# Predictive Maintenance ML Training Lab
## IoT Screw Machine - Drill Bit Replacement Prediction

---

### **Lab Overview**

This notebook demonstrates an end-to-end Machine Learning pipeline for **predictive maintenance** using IoT sensor data from industrial screw machines. The goal is to predict when drill bits need replacement, enabling proactive maintenance and preventing unexpected downtime.

---

### **Business Problem**

Manufacturing operations face significant challenges:
- **Unplanned downtime** costs up to $260,000/hour in automotive manufacturing
- **Reactive maintenance** leads to emergency repairs and rushed part orders
- **Scheduled maintenance** wastes resources by replacing parts too early
- **Manual monitoring** cannot scale across hundreds of machines

**Solution:** Use historical sensor data and machine learning to predict the optimal replacement date for each drill bit.

---

### **Dataset**

**Source:** IoT telemetry from screw machines with drill bit usage tracking

**Key Metrics:**
- `BitRotationCount`: Cumulative rotations since last bit replacement
- `Timestamp`: Date/time of each telemetry record
- `MachineID`: Unique identifier for each machine
- Additional sensor data: Torque, Temperature, Vibration, etc.

**Critical Pattern:** When a drill bit is replaced, the `BitRotationCount` resets to zero, creating natural "cycles" we can learn from.

---

### **ML Approach**

**Model Type:** XGBoost Regression (Gradient Boosting Decision Trees)

**Target Variable:** `RemainingRotations` - rotations left until next bit replacement

**Features:**
- Current cumulative rotations
- Time-based features (hour, day of week)
- Rolling window statistics (last 6 hours activity)
- Rotation rate and acceleration
- Cycle progress indicators

**Training Strategy:**
- Detect replacement events via counter resets
- Segment data into individual replacement cycles
- Calculate remaining useful life for each timestamp
- Train on completed cycles, validate on recent cycles
- Convert rotation predictions to calendar dates

---

### **Production Deployment**

**Daily Scoring Pipeline:**
1. Load latest telemetry from Fabric Lakehouse
2. Engineer features for current machine states
3. Predict remaining rotations using trained model
4. Convert to predicted replacement dates
5. Classify risk levels (CRITICAL < 1 day, HIGH < 3 days, etc.)
6. Write predictions to Lakehouse table
7. Trigger Power Automate alerts for critical machines

**Integration:**
- **Power BI Dashboard:** Real-time monitoring with Gantt chart timeline
- **Power Automate:** Email/Teams alerts for critical predictions
- **Power Apps:** Work order management system for maintenance engineers

---

### **Expected Outcomes**

✅ **Predict replacement dates** with 1-3 day accuracy  
✅ **Identify critical machines** requiring immediate attention  
✅ **Reduce unplanned downtime** by 40-60%  
✅ **Optimize maintenance scheduling** and parts inventory  
✅ **Enable proactive maintenance** culture shift

---

### **Notebook Structure**

This lab is organized into **15 sequential parts:**

| Part | Description | Output |
|------|-------------|--------|
| **Part 1** | Setup and Data Loading | Load CSV, inspect data |
| **Part 2** | Replacement Event Detection | Detect counter resets, create cycles |
| **Part 3** | Cycle Segmentation | Assign cycle IDs, calculate cycle metrics |
| **Part 4** | Feature Engineering | Rolling windows, time features, rates |
| **Part 5** | Target Variable Creation | Calculate `RemainingRotations` |
| **Part 6** | Model Training (XGBoost) | Train regression model, track with MLflow |
| **Part 7** | Model Evaluation | MAE, R², feature importance |
| **Part 8** | Prediction Generation | Score current machine states |
| **Part 9** | Rotation Rate Estimation | Calculate rotations/hour for each machine |
| **Part 10** | Date Conversion | Convert rotations → predicted dates |
| **Part 11** | Production Pipeline | Daily scoring workflow |
| **Part 12** | Power BI Integration | DAX measures, Gantt chart setup |
| **Part 13** | Power Automate Alerts | Email notifications, work order creation |
| **Part 14** | Power Apps Work Orders | Canvas app for maintenance engineers |
| **Part 15** | Model Monitoring | Performance tracking, retraining strategy |

---

### **Prerequisites**

**Environment:**
- Microsoft Fabric workspace with Lakehouse
- Python 3.10+ with Spark runtime
- Power BI Premium or Fabric capacity

**Licensing (for Parts 12-14):**
- **Microsoft Fabric**: F64 capacity or higher (includes Power BI Premium)
- **Power Automate**: 
  - Included with Microsoft 365 (for standard connectors)
  - Premium license required for Dataverse connector (Part 13-14)
- **Power Apps**:
  - **Option A (Dataverse)**: Power Apps premium license (~$20/user/month)
  - **Option B (SharePoint)**: Included with Microsoft 365 (E3/E5 or Business Premium)
- **Microsoft Dataverse**: 
  - Included with Power Apps premium (Option A only)
  - Database capacity: 10 GB included, additional storage available
- **Microsoft 365**: E3, E5, or Business Premium
  - Includes: Office 365 Outlook, SharePoint Online, Microsoft Teams
- **Microsoft Teams**: Included with Microsoft 365 (for notifications)

**License Summary by Part:**
- Parts 1-11: Microsoft Fabric only
- Part 12: Power BI (included in Fabric)
- Part 13: Power Automate + Office 365 Outlook + Dataverse (premium) or SharePoint (M365)
- Part 14: Power Apps premium (Dataverse) OR Power Apps standard (SharePoint with M365)
- Part 15: Microsoft Fabric only

💡 **Cost-Effective Option:** Use SharePoint List backend (Option B in Parts 13-14) to avoid Power Apps premium license - only requires Microsoft 365 E3/Business Premium.

**Libraries:**
- `pandas`, `numpy` - Data manipulation
- `scikit-learn` - ML utilities and evaluation
- `xgboost` - Gradient boosting model
- `mlflow` - Experiment tracking
- `matplotlib`, `seaborn` - Visualization

**Data Requirements:**
- Historical IoT telemetry data (minimum 30 days)
- At least 3-5 completed replacement cycles per machine
- Reliable timestamp and rotation counter fields

---

### **Lab Instructions**

⚠️ **CRITICAL:** Run cells sequentially from Part 1 → Part 15. Do not skip or reorder cells.

Each part builds on the previous parts. Skipping cells will cause `KeyError` or missing variable exceptions.

💡 **Tips:**
- Read markdown cells carefully before running code
- Check outputs after each cell to verify correctness
- Use `df.head()` to inspect data at any stage
- Adjust parameters (like thresholds) based on your data

---

**Ready to start? Begin with Part 1 below. ⬇️**

---

## Part 1: Setup and Data Loading

⚠️ **IMPORTANT: Run cells in sequential order (Part 1 → Part 2 → Part 3 → etc.)**

Each part depends on the previous parts. If you skip cells or run out of order, you'll get `KeyError` exceptions.

**Cell Dependencies:**
- Part 3 requires Part 2 (needs `CumulativeBitRotation`, `cycle_id`)
- Part 4 requires Part 3 (needs `cycle_start_time`)
- Part 5+ require Parts 2-4 (need all engineered features)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

In [ ]:
# Load CSV from local file or Lakehouse Files
# OPTION 1: Load from local file (if running in Fabric with uploaded file)
csv_path = "Files/quality_30days.csv"

# OPTION 2: Load from local path (if CSV is in same directory as notebook)
# csv_path = "quality_30days.csv"

# OPTION 3: Load from sample_quality_data.csv (generated by generate_historical_data.py)
# csv_path = "sample_quality_data.csv"

print(f"📂 Attempting to load: {csv_path}")

try:
    # Try loading from Lakehouse Files first
    df_spark = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(csv_path)
    print(f"✅ Loaded from Lakehouse Files")
except Exception as e:
    print(f"⚠️ Could not load from Lakehouse Files: {str(e)[:100]}")
    print(f"\n💡 To fix this:")
    print(f"1. In Fabric Lakehouse, go to your 'MaintenanceML' lakehouse")
    print(f"2. Click 'Upload' → 'Upload files'")
    print(f"3. Select 'Files' folder (not 'Tables')")
    print(f"4. Upload your 'sample_quality_data.csv' or 'quality_30days.csv'")
    print(f"\nAlternatively, load from local pandas:")
    
    # Fallback: Try loading from local file using pandas
    import os
    local_files = ['sample_quality_data.csv', 'quality_30days.csv', '../sample_quality_data.csv']
    
    df = None
    for local_path in local_files:
        if os.path.exists(local_path):
            print(f"📂 Found local file: {local_path}")
            df = pd.read_csv(local_path)
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            
            # Convert to Spark and save to Lakehouse
            df_spark = spark.createDataFrame(df)
            print(f"✅ Loaded from local file: {local_path}")
            break
    
    if df is None:
        raise FileNotFoundError(
            f"❌ CSV file not found!\n"
            f"Please ensure you have:\n"
            f"1. Generated data using: python generate_historical_data.py --days 30 --interval 1 --output quality_30days.csv\n"
            f"2. Uploaded the CSV to Fabric Lakehouse Files folder, OR\n"
            f"3. Have the CSV file in the current directory"
        )

# Save to Lakehouse table
df_spark.write.mode("overwrite").saveAsTable("MaintenanceML.machine_data_raw")
print(f"✅ Saved to table: MaintenanceML.machine_data_raw")

# Convert to pandas for processing
df = df_spark.toPandas()
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(['MachineID', 'Timestamp']).reset_index(drop=True)

print(f"\n✅ Loaded {len(df):,} records")
print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
print(f"Machines: {df['MachineID'].nunique()}")
print(f"\nColumns: {df.columns.tolist()}")

## Part 2: Detect Replacement Events (Counter Resets)

**Logic:** A replacement event occurs when CumulativeBitRotation decreases or resets to near-zero.

We detect:
- Large negative jumps in counter (delta < 0)
- Counter drops more than threshold (e.g., -1000 rotations)

In [ ]:
# Calculate rotation features first
df['RotationCount'] = df['ActualAngle'] / 360.0

# Use BitRotationCounter if available, otherwise cumulative rotation
if 'BitRotationCounter' in df.columns:
    df['CumulativeBitRotation'] = df['BitRotationCounter']
else:
    df['CumulativeBitRotation'] = df.groupby('MachineID')['RotationCount'].cumsum()

print(f"✅ Rotation features calculated")
print(f"Max cumulative rotation: {df['CumulativeBitRotation'].max():,.0f}")

In [ ]:
# Detect replacement events (counter resets)
df['delta_counter'] = df.groupby('MachineID')['CumulativeBitRotation'].diff()

# Replacement detected when counter decreases (negative delta) or large drop
RESET_THRESHOLD = -1000  # rotations
df['is_reset'] = (df['delta_counter'] < RESET_THRESHOLD) | (df['delta_counter'].isna())

# Assign cycle_id (increments at each replacement)
df['cycle_id'] = df.groupby('MachineID')['is_reset'].cumsum()

# Rotation count within current cycle (resets to current value after replacement)
df['rotation_in_cycle'] = df.groupby(['MachineID', 'cycle_id'])['CumulativeBitRotation'].transform(
    lambda x: x - x.iloc[0] + x.iloc[0]  # Keep actual counter value per cycle
)

print(f"✅ Replacement events detected")
print(f"Total replacement events: {df['is_reset'].sum():,}")
print(f"Total cycles detected: {df.groupby(['MachineID', 'cycle_id']).ngroups:,}")

# Show replacement events
reset_events = df[df['is_reset']].copy()
if len(reset_events) > 0:
    print(f"\n🔧 Sample Replacement Events:")
    print(reset_events[['Timestamp', 'MachineID', 'CumulativeBitRotation', 'delta_counter', 'cycle_id']].head(10))

## Part 3: Create Labels - Remaining Rotations Until Next Replacement

**Target Variable:** `RemainingRotations` = rotations left until next replacement

For each cycle, we find:
- `cycle_end_rotation`: Maximum rotation reached before replacement
- `cycle_end_time`: Timestamp of replacement
- `RemainingRotations_t = cycle_end_rotation - rotation_in_cycle_t`

In [ ]:
# Calculate cycle statistics (end rotation and end time per cycle)
# Note: This requires Part 2 to be run first (CumulativeBitRotation and cycle_id must exist)

# Verify required columns exist
required_cols = ['CumulativeBitRotation', 'cycle_id', 'MachineID', 'Timestamp']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}. Please run Part 2 first!")

cycle_stats = df.groupby(['MachineID', 'cycle_id']).agg(
    cycle_end_rotation=('CumulativeBitRotation', 'max'),
    cycle_end_time=('Timestamp', 'max'),
    cycle_start_time=('Timestamp', 'min'),
    cycle_record_count=('Timestamp', 'count')
).reset_index()

# Calculate cycle duration in days
cycle_stats['cycle_duration_days'] = (cycle_stats['cycle_end_time'] - cycle_stats['cycle_start_time']).dt.total_seconds() / 86400

print(f"✅ Cycle statistics calculated")
print(f"\n📊 Cycle Summary:")
print(cycle_stats[['MachineID', 'cycle_id', 'cycle_end_rotation', 'cycle_duration_days', 'cycle_record_count']].head(10))
print(f"\nAverage cycle duration: {cycle_stats['cycle_duration_days'].mean():.1f} days")
print(f"Average rotations per cycle: {cycle_stats['cycle_end_rotation'].mean():,.0f}")

In [ ]:
# Merge cycle stats back to main dataframe
print(f"📊 Merging cycle stats...")
print(f"Main df shape before merge: {df.shape}")
print(f"Cycle stats shape: {cycle_stats.shape}")
print(f"Cycle stats columns: {cycle_stats.columns.tolist()}")

df = df.merge(cycle_stats[['MachineID', 'cycle_id', 'cycle_end_rotation', 'cycle_end_time', 'cycle_start_time']], 
              on=['MachineID', 'cycle_id'], how='left')

print(f"Main df shape after merge: {df.shape}")
print(f"Columns after merge: {df.columns.tolist()}")

# Calculate remaining rotations (target variable)
df['RemainingRotations'] = df['cycle_end_rotation'] - df['CumulativeBitRotation']

# Calculate remaining days (derived from time difference)
df['RemainingDays_Actual'] = (df['cycle_end_time'] - df['Timestamp']).dt.total_seconds() / 86400

# Clip negative values (edge cases)
df['RemainingRotations'] = df['RemainingRotations'].clip(lower=0)
df['RemainingDays_Actual'] = df['RemainingDays_Actual'].clip(lower=0)

# Calculate time since cycle start (helpful feature)
df['time_since_cycle_start_hours'] = (df['Timestamp'] - df['cycle_start_time']).dt.total_seconds() / 3600

print(f"✅ Target labels created: RemainingRotations")
print(f"\n🎯 Sample Labels:")
print(df[['Timestamp', 'MachineID', 'cycle_id', 'CumulativeBitRotation', 'RemainingRotations', 'RemainingDays_Actual']].head(20))

## Part 4: Feature Engineering - Rolling Windows and Aggregates

**Features to capture:**
- Rotations per hour (recent activity rate)
- Rolling statistics: torque, cycle time, NG rate
- Time features: hour of day, day of week
- Current position in cycle

In [ ]:
# Set timestamp as index for rolling calculations (make a copy to preserve original df)
df_indexed = df.copy().set_index('Timestamp')

# Verify required columns exist from Part 3
required_cols_part4 = ['Timestamp', 'MachineID', 'RotationCount', 'ActualTorque', 'CycleTime_ms', 'CycleOK']
missing_cols_part4 = [col for col in required_cols_part4 if col not in df.columns]
if missing_cols_part4:
    raise ValueError(f"Missing required columns: {missing_cols_part4}. Please run Parts 2-3 first!")

# Create rolling window features (1 hour, 4 hours)
rolling_1h = df_indexed.groupby('MachineID').rolling('1H').agg({
    'RotationCount': ['sum', 'mean'],
    'ActualTorque': ['mean', 'std'],
    'CycleTime_ms': ['mean', 'max'],
    'CycleOK': 'mean'  # Pass rate
}).reset_index()

rolling_1h.columns = ['MachineID', 'Timestamp', 
                      'Rotations_LastHour', 'Rotations_LastHour_Avg',
                      'AvgTorque_LastHour', 'StdTorque_LastHour',
                      'CycleTime_Avg_LastHour', 'CycleTime_Max_LastHour',
                      'PassRate_LastHour']

# Rolling 4 hours
rolling_4h = df_indexed.groupby('MachineID').rolling('4H').agg({
    'RotationCount': 'sum',
    'CycleOK': 'mean'
}).reset_index()

rolling_4h.columns = ['MachineID', 'Timestamp', 'Rotations_Last4Hours', 'PassRate_Last4Hours']

# Debug: Show merge keys and shapes
print(f"📊 Merging rolling features...")
print(f"Main df shape: {df.shape}, has Timestamp column: {'Timestamp' in df.columns}")
print(f"Rolling 1H shape: {rolling_1h.shape}")
print(f"Rolling 4H shape: {rolling_4h.shape}")

# Merge rolling features (use outer join to catch merge issues)
df = df.merge(rolling_1h, on=['MachineID', 'Timestamp'], how='left')
df = df.merge(rolling_4h, on=['MachineID', 'Timestamp'], how='left')

# Verify merge succeeded
if 'Rotations_LastHour' not in df.columns:
    raise ValueError("Rolling feature merge failed! Check if Timestamp column exists and matches between dataframes.")

# Fill NaN with 0 for first hours
df = df.fillna(0)

print(f"✅ Rolling window features created")
print(f"Final df shape: {df.shape}")
print(f"\n📈 Sample Rolling Features:")
print(df[['MachineID', 'Rotations_LastHour', 'AvgTorque_LastHour', 'PassRate_LastHour']].head(10))

In [ ]:
# Add time-based features
df['hour_of_day'] = df['Timestamp'].dt.hour
df['day_of_week'] = df['Timestamp'].dt.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

# Calculate rotation rate (rotations per minute)
df['rotation_rate_per_min'] = df['Rotations_LastHour'] / 60.0

# NG rate over last 100 cycles (if available)
if 'CycleOK' in df.columns:
    df['NG_Rate_Last100'] = 1 - df.groupby('MachineID')['CycleOK'].transform(lambda x: x.rolling(100, min_periods=1).mean())
else:
    df['NG_Rate_Last100'] = 0

print(f"✅ Time-based and rate features created")
print(f"\n🕐 Sample Time Features:")
print(df[['Timestamp', 'hour_of_day', 'day_of_week', 'rotation_rate_per_min', 'NG_Rate_Last100']].head(10))

## Part 5: Prepare Training Dataset

**Key points:**
- Exclude incomplete cycles (no observed replacement yet) from training
- Use time-based split (not random) to avoid temporal leakage
- Train on older cycles, validate on recent cycles

In [ ]:
# Filter out incomplete cycles (where cycle_end_time is NaT)
# Verify required columns exist from Part 3
required_cols_part5 = ['cycle_end_time', 'RemainingRotations', 'RemainingDays_Actual']
missing_cols_part5 = [col for col in required_cols_part5 if col not in df.columns]
if missing_cols_part5:
    raise ValueError(f"Missing required columns: {missing_cols_part5}. Please run Part 3 first!")

train_df = df[~df['cycle_end_time'].isna()].copy()

print(f"✅ Training dataset prepared")
print(f"Total records: {len(df):,}")
print(f"Training records (complete cycles): {len(train_df):,}")
print(f"Incomplete cycles excluded: {len(df) - len(train_df):,}")

# Check for valid target
print(f"\nTarget variable (RemainingRotations):")
print(f"  Min: {train_df['RemainingRotations'].min():,.0f}")
print(f"  Max: {train_df['RemainingRotations'].max():,.0f}")
print(f"  Mean: {train_df['RemainingRotations'].mean():,.0f}")
print(f"  Median: {train_df['RemainingRotations'].median():,.0f}")

In [ ]:
# Define feature columns for ML model
feature_cols = [
    # Core features
    'CumulativeBitRotation',
    'rotation_in_cycle',
    'time_since_cycle_start_hours',
    
    # Rolling activity features
    'Rotations_LastHour',
    'Rotations_Last4Hours',
    'rotation_rate_per_min',
    
    # Quality features
    'AvgTorque_LastHour',
    'StdTorque_LastHour',
    'CycleTime_Avg_LastHour',
    'CycleTime_Max_LastHour',
    'PassRate_LastHour',
    'PassRate_Last4Hours',
    'NG_Rate_Last100',
    
    # Time features
    'hour_of_day',
    'day_of_week',
    'is_weekend',
    
    # Current state
    'ActualTorque',
    'ActualAngle',
    'CycleTime_ms'
]

# Verify all features exist
available_features = [col for col in feature_cols if col in train_df.columns]
missing_features = [col for col in feature_cols if col not in train_df.columns]

if missing_features:
    print(f"⚠️ Missing features (will be excluded): {missing_features}")
    feature_cols = available_features

print(f"\n✅ Feature set prepared: {len(feature_cols)} features")
print(f"Features: {feature_cols}")

# Prepare X and y
X_train_full = train_df[feature_cols]
y_train_full = train_df['RemainingRotations']

print(f"\n📊 Training set shape: X={X_train_full.shape}, y={y_train_full.shape}")

In [ ]:
# Time-based train/validation split (80/20)
# Use last 20% of cycles for validation
split_idx = int(len(train_df) * 0.8)

X_train = X_train_full.iloc[:split_idx]
X_val = X_train_full.iloc[split_idx:]
y_train = y_train_full.iloc[:split_idx]
y_val = y_train_full.iloc[split_idx:]

print(f"✅ Train/Validation split completed")
print(f"Training set: {len(X_train):,} records")
print(f"Validation set: {len(X_val):,} records")
print(f"Split ratio: {len(X_train)/len(X_train_full)*100:.1f}% train, {len(X_val)/len(X_train_full)*100:.1f}% validation")

## Part 6: Train XGBoost Model with MLflow Tracking

**Model:** XGBoost Regressor for `RemainingRotations` prediction

**Why XGBoost:**
- Excellent for tabular data with non-linear relationships
- Fast training and inference
- Handles missing values
- Explainable with SHAP values

In [ ]:
# Train XGBoost model with MLflow tracking
import mlflow
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set up MLflow experiment
EXPERIMENT_NAME = "Replacement_Date_Prediction"
mlflow.set_experiment(EXPERIMENT_NAME)

# Prepare DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'eval_metric': 'mae',
    'seed': 12345
}

# Start MLflow run
with mlflow.start_run(run_name="XGBoost_ReplacementPrediction_v1") as run:
    # Train model with early stopping
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtrain, 'train'), (dval, 'val')],
        early_stopping_rounds=50,
        verbose_eval=50
    )
    
    # Make predictions
    y_train_pred = model.predict(dtrain)
    y_val_pred = model.predict(dval)
    
    # Calculate metrics
    train_mae = mean_absolute_error(y_train, y_train_pred)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    
    # Log parameters
    mlflow.log_params(params)
    mlflow.log_param("num_features", len(feature_cols))
    mlflow.log_param("train_samples", len(X_train))
    mlflow.log_param("val_samples", len(X_val))
    
    # Log metrics
    mlflow.log_metric("train_mae_rotations", train_mae)
    mlflow.log_metric("val_mae_rotations", val_mae)
    mlflow.log_metric("train_rmse_rotations", train_rmse)
    mlflow.log_metric("val_rmse_rotations", val_rmse)
    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("val_r2", val_r2)
    
    # Log model
    mlflow.xgboost.log_model(model, "xgboost_model")
    
    # Save model locally
    model.save_model('replacement_prediction_model.json')
    
    print(f"\n✅ Model training completed")
    print(f"\n📊 Training Metrics:")
    print(f"  MAE (rotations): {train_mae:,.0f}")
    print(f"  RMSE (rotations): {train_rmse:,.0f}")
    print(f"  R²: {train_r2:.4f}")
    print(f"\n📊 Validation Metrics:")
    print(f"  MAE (rotations): {val_mae:,.0f}")
    print(f"  RMSE (rotations): {val_rmse:,.0f}")
    print(f"  R²: {val_r2:.4f}")
    print(f"\n✅ Model saved to MLflow: {run.info.run_id}")

## Part 7: Feature Importance Analysis

Understanding which features drive replacement predictions

In [ ]:
# Get feature importance
import matplotlib.pyplot as plt

# Get importance scores
importance_dict = model.get_score(importance_type='gain')
feature_importance = pd.DataFrame([
    {'feature': k, 'importance': v} 
    for k, v in importance_dict.items()
]).sort_values('importance', ascending=False)

# Map feature indices back to names
feature_importance['feature_name'] = feature_importance['feature'].apply(
    lambda x: feature_cols[int(x.replace('f', ''))] if x.startswith('f') else x
)

print(f"✅ Feature importance calculated")
print(f"\n📊 Top 10 Most Important Features:")
print(feature_importance[['feature_name', 'importance']].head(10).to_string(index=False))

# Plot top features
top_features = feature_importance.head(15)
plt.figure(figsize=(10, 6))
plt.barh(top_features['feature_name'], top_features['importance'])
plt.xlabel('Importance (Gain)')
plt.title('Top 15 Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Log feature importance to MLflow
with mlflow.start_run(run_id=run.info.run_id):
    mlflow.log_figure(plt.gcf(), "feature_importance.png")

## Part 8: Generate Predictions and Convert to Replacement Date

**Steps:**
1. Predict `RemainingRotations` for current state
2. Estimate `RotationsPerHour` from recent activity
3. Calculate `PredictedDays = RemainingRotations / (RotationsPerHour * 24)`
4. Calculate `PredictedReplacementDate = Now + PredictedDays`

In [ ]:
# Get latest state per machine (most recent timestamp)
df_latest = df.sort_values('Timestamp').groupby('MachineID').tail(1).reset_index(drop=True)

# Prepare features for prediction
X_latest = df_latest[feature_cols]
dlatest = xgb.DMatrix(X_latest)

# Make predictions
pred_remaining_rotations = model.predict(dlatest)

# Add predictions to dataframe
df_latest['Pred_RemainingRotations'] = pred_remaining_rotations

print(f"✅ Predictions generated for {len(df_latest)} machines")
print(f"\n🔮 Predicted Remaining Rotations:")
print(df_latest[['MachineID', 'CumulativeBitRotation', 'Pred_RemainingRotations']].head(10))

In [ ]:
# Convert predictions to days and replacement date

# Estimate rotations per hour from recent activity
df_latest['RotationsPerHour_Estimate'] = df_latest['Rotations_LastHour']

# Handle edge case: if no recent activity, use global average
global_avg_rph = train_df[train_df['Rotations_LastHour'] > 0]['Rotations_LastHour'].mean()
df_latest['RotationsPerHour_Estimate'] = df_latest['RotationsPerHour_Estimate'].replace(0, global_avg_rph)

# Calculate predicted days
df_latest['Pred_DaysLeft'] = (df_latest['Pred_RemainingRotations'] / df_latest['RotationsPerHour_Estimate']) / 24.0

# Clip to reasonable range (0-365 days)
df_latest['Pred_DaysLeft'] = df_latest['Pred_DaysLeft'].clip(0, 365)

# Calculate predicted replacement date
current_time = pd.Timestamp.now()
df_latest['Pred_ReplacementDate'] = current_time + pd.to_timedelta(df_latest['Pred_DaysLeft'], unit='D')

# Add prediction timestamp
df_latest['PredictionTimestamp'] = current_time

print(f"✅ Replacement dates calculated")
print(f"\n📅 Predicted Replacement Dates:")
print(df_latest[['MachineID', 'Pred_DaysLeft', 'Pred_ReplacementDate']].head(10).to_string(index=False))

In [ ]:
# Calculate risk levels based on predicted days
def assign_risk_level(days):
    if days <= (1/24):  # <= 1 hour
        return '🔴 CRITICAL'
    elif days <= 1:  # <= 1 day
        return '🟠 HIGH'
    elif days <= 7:  # <= 7 days
        return '🟡 MEDIUM'
    else:
        return '🟢 LOW'

df_latest['RiskLevel'] = df_latest['Pred_DaysLeft'].apply(assign_risk_level)
df_latest['IsCritical'] = (df_latest['Pred_DaysLeft'] <= (1/24)).astype(int)

print(f"✅ Risk levels assigned")
print(f"\n🚨 Risk Distribution:")
print(df_latest['RiskLevel'].value_counts())
print(f"\n⚠️ Critical machines (< 1 hour): {df_latest['IsCritical'].sum()}")

## Part 9: Save Predictions to Lakehouse

Output table: `machine.predictions`

Columns:
- PredictionTimestamp
- MachineID
- CumulativeBitRotation
- rotation_in_cycle
- Pred_RemainingRotations
- Pred_DaysLeft
- Pred_ReplacementDate
- RiskLevel
- IsCritical
- ModelVersion

In [ ]:
# Prepare prediction output table
prediction_output_cols = [
    'PredictionTimestamp',
    'Timestamp',
    'MachineID',
    'CumulativeBitRotation',
    'rotation_in_cycle',
    'Rotations_LastHour',
    'RotationsPerHour_Estimate',
    'Pred_RemainingRotations',
    'Pred_DaysLeft',
    'Pred_ReplacementDate',
    'RiskLevel',
    'IsCritical'
]

df_predictions = df_latest[prediction_output_cols].copy()
df_predictions['ModelVersion'] = 'v1_xgboost'

# Convert to Spark and save to Lakehouse
spark_predictions = spark.createDataFrame(df_predictions)
spark_predictions.write.mode("overwrite").saveAsTable("MaintenanceML.replacement_predictions")

print(f"✅ Predictions saved to: MaintenanceML.replacement_predictions")
print(f"Total predictions: {len(df_predictions)}")
print(f"\n📊 Sample Predictions:")
print(df_predictions[['MachineID', 'Pred_DaysLeft', 'Pred_ReplacementDate', 'RiskLevel']].head(10).to_string(index=False))

## Part 10: Prediction Comparison - ML vs Actual

Compare predicted vs actual remaining days on validation set

In [ ]:
# Get validation set predictions
val_df = train_df.iloc[split_idx:].copy()
val_df['Pred_RemainingRotations'] = y_val_pred

# 🔍 DIAGNOSTIC: Check rotation predictions before conversion
print(f"\n🔍 Diagnostic: Rotation Predictions")
print(f"Pred_RemainingRotations - Min: {val_df['Pred_RemainingRotations'].min():,.0f}")
print(f"Pred_RemainingRotations - Max: {val_df['Pred_RemainingRotations'].max():,.0f}")
print(f"Pred_RemainingRotations - Mean: {val_df['Pred_RemainingRotations'].mean():,.0f}")
print(f"Actual RemainingRotations - Mean: {val_df['RemainingRotations'].mean():,.0f}")

# 🔍 DIAGNOSTIC: Check rotation rate used
print(f"\n🔍 Diagnostic: Rotation Rates")
print(f"Rotations_LastHour - Min: {val_df['Rotations_LastHour'].min():,.2f}")
print(f"Rotations_LastHour - Max: {val_df['Rotations_LastHour'].max():,.2f}")
print(f"Rotations_LastHour - Mean: {val_df['Rotations_LastHour'].mean():,.2f}")

# 🎯 FIX: Use cycle-average rotation rate instead of instantaneous last hour
# Calculate average rotations per day for each cycle (more stable near cycle end)
val_df['RotationsPerDay_Avg'] = val_df.groupby(['MachineID', 'cycle_id'])['Rotations_LastHour'].transform(
    lambda x: x.mean() * 24
)

# Use average rate for conversion (more robust than instantaneous rate)
val_df['Pred_DaysLeft'] = val_df['Pred_RemainingRotations'] / val_df['RotationsPerDay_Avg'].clip(lower=1)

# 🔍 DIAGNOSTIC: Check predicted days BEFORE clipping
print(f"\n🔍 Diagnostic: Predicted Days (before clip)")
print(f"Pred_DaysLeft - Min: {val_df['Pred_DaysLeft'].min():,.2f}")
print(f"Pred_DaysLeft - Max: {val_df['Pred_DaysLeft'].max():,.2f}")
print(f"Pred_DaysLeft - Mean: {val_df['Pred_DaysLeft'].mean():,.2f}")
print(f"Pred_DaysLeft - % > 365: {(val_df['Pred_DaysLeft'] > 365).mean() * 100:.1f}%")

val_df['Pred_DaysLeft'] = val_df['Pred_DaysLeft'].clip(0, 365)

# Calculate MAE in days
mae_days = mean_absolute_error(val_df['RemainingDays_Actual'], val_df['Pred_DaysLeft'])

print(f"✅ Validation predictions converted to days")
print(f"\n📊 Validation Metrics (Days):")
print(f"  MAE: {mae_days:.2f} days")
print(f"  RMSE: {np.sqrt(mean_squared_error(val_df['RemainingDays_Actual'], val_df['Pred_DaysLeft'])):.2f} days")
print(f"  R²: {r2_score(val_df['RemainingDays_Actual'], val_df['Pred_DaysLeft']):.4f}")

# Show sample comparisons
comparison = val_df[['MachineID', 'Timestamp', 'RemainingDays_Actual', 'Pred_DaysLeft']].head(20)
comparison['Error_Days'] = comparison['Pred_DaysLeft'] - comparison['RemainingDays_Actual']
print(f"\n🔍 Sample Predictions vs Actual:")
print(comparison.to_string(index=False))

In [ ]:
# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(val_df['RemainingDays_Actual'], val_df['Pred_DaysLeft'], alpha=0.3)
plt.plot([0, val_df['RemainingDays_Actual'].max()], [0, val_df['RemainingDays_Actual'].max()], 'r--', label='Perfect Prediction')
plt.xlabel('Actual Remaining Days')
plt.ylabel('Predicted Remaining Days')
plt.title(f'Prediction vs Actual (MAE: {mae_days:.2f} days)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Log to MLflow
with mlflow.start_run(run_id=run.info.run_id):
    mlflow.log_figure(plt.gcf(), "prediction_vs_actual.png")
    mlflow.log_metric("val_mae_days", mae_days)

## Part 11: Daily Scoring Pipeline (Production)

**Use this cell for scheduled daily predictions**

Schedule in Fabric Data Pipeline:
1. Create Data Pipeline: `Daily_Replacement_Predictions`
2. Add Notebook activity → Select this notebook
3. Set schedule: Daily at 1:00 AM UTC
4. Publish

In [ ]:
# Daily automated prediction workflow
# Run this cell for scheduled updates with new data

def daily_prediction_update():
    """
    Daily scoring pipeline:
    1. Load latest data from Lakehouse
    2. Apply feature engineering
    3. Load trained model
    4. Generate predictions
    5. Save to predictions table (append mode)
    """
    
    print(f"🚀 Starting daily prediction update: {datetime.now()}")
    
    # Step 1: Load latest data
    df_new = spark.read.table("MaintenanceML.machine_data_raw").toPandas()
    df_new['Timestamp'] = pd.to_datetime(df_new['Timestamp'])
    df_new = df_new.sort_values(['MachineID', 'Timestamp']).reset_index(drop=True)
    
    print(f"✅ Loaded {len(df_new):,} records")
    
    # Step 2: Apply feature engineering (Parts 2-4)
    
    # Part 2: Calculate rotation features
    df_new['RotationCount'] = df_new['ActualAngle'] / 360.0
    
    if 'BitRotationCounter' in df_new.columns:
        df_new['CumulativeBitRotation'] = df_new['BitRotationCounter']
    else:
        df_new['CumulativeBitRotation'] = df_new.groupby('MachineID')['RotationCount'].cumsum()
    
    # Detect replacement events
    df_new['delta_counter'] = df_new.groupby('MachineID')['CumulativeBitRotation'].diff()
    RESET_THRESHOLD = -1000
    df_new['is_reset'] = (df_new['delta_counter'] < RESET_THRESHOLD) | (df_new['delta_counter'].isna())
    df_new['cycle_id'] = df_new.groupby('MachineID')['is_reset'].cumsum()
    
    df_new['rotation_in_cycle'] = df_new.groupby(['MachineID', 'cycle_id'])['CumulativeBitRotation'].transform(
        lambda x: x - x.iloc[0] + x.iloc[0]
    )
    
    # Part 3: Cycle statistics (for time_since_cycle_start)
    cycle_stats = df_new.groupby(['MachineID', 'cycle_id']).agg(
        cycle_start_time=('Timestamp', 'min')
    ).reset_index()
    
    df_new = df_new.merge(cycle_stats[['MachineID', 'cycle_id', 'cycle_start_time']], 
                          on=['MachineID', 'cycle_id'], how='left')
    
    df_new['time_since_cycle_start_hours'] = (df_new['Timestamp'] - df_new['cycle_start_time']).dt.total_seconds() / 3600
    
    # Part 4: Rolling window features
    df_indexed = df_new.set_index('Timestamp')
    
    rolling_1h = df_indexed.groupby('MachineID').rolling('1H').agg({
        'RotationCount': ['sum', 'mean'],
        'ActualTorque': ['mean', 'std'],
        'CycleTime_ms': ['mean', 'max'],
        'CycleOK': 'mean'
    }).reset_index()
    
    rolling_1h.columns = ['MachineID', 'Timestamp', 
                          'Rotations_LastHour', 'Rotations_LastHour_Avg',
                          'AvgTorque_LastHour', 'StdTorque_LastHour',
                          'CycleTime_Avg_LastHour', 'CycleTime_Max_LastHour',
                          'PassRate_LastHour']
    
    rolling_4h = df_indexed.groupby('MachineID').rolling('4H').agg({
        'RotationCount': 'sum',
        'CycleOK': 'mean'
    }).reset_index()
    
    rolling_4h.columns = ['MachineID', 'Timestamp', 'Rotations_Last4Hours', 'PassRate_Last4Hours']
    
    df_new = df_new.merge(rolling_1h, on=['MachineID', 'Timestamp'], how='left')
    df_new = df_new.merge(rolling_4h, on=['MachineID', 'Timestamp'], how='left')
    df_new = df_new.fillna(0)
    
    # Time-based features
    df_new['hour_of_day'] = df_new['Timestamp'].dt.hour
    df_new['day_of_week'] = df_new['Timestamp'].dt.dayofweek
    df_new['is_weekend'] = (df_new['day_of_week'] >= 5).astype(int)
    df_new['rotation_rate_per_min'] = df_new['Rotations_LastHour'] / 60.0
    
    if 'CycleOK' in df_new.columns:
        df_new['NG_Rate_Last100'] = 1 - df_new.groupby('MachineID')['CycleOK'].transform(
            lambda x: x.rolling(100, min_periods=1).mean()
        )
    else:
        df_new['NG_Rate_Last100'] = 0
    
    print(f"✅ Feature engineering completed")
    
    # Step 3: Load trained model
    model_loaded = xgb.Booster()
    model_loaded.load_model('replacement_prediction_model.json')
    
    print(f"✅ Model loaded")
    
    # Step 4: Generate predictions (latest state per machine)
    df_latest_new = df_new.sort_values('Timestamp').groupby('MachineID').tail(1).reset_index(drop=True)
    
    # Select features (same as training)
    feature_cols_predict = [
        'CumulativeBitRotation', 'rotation_in_cycle', 'time_since_cycle_start_hours',
        'Rotations_LastHour', 'Rotations_Last4Hours', 'rotation_rate_per_min',
        'AvgTorque_LastHour', 'StdTorque_LastHour', 'CycleTime_Avg_LastHour', 
        'CycleTime_Max_LastHour', 'PassRate_LastHour', 'PassRate_Last4Hours',
        'NG_Rate_Last100', 'hour_of_day', 'day_of_week', 'is_weekend',
        'ActualTorque', 'ActualAngle', 'CycleTime_ms'
    ]
    
    X_latest_new = df_latest_new[feature_cols_predict]
    dlatest_new = xgb.DMatrix(X_latest_new)
    
    pred_remaining_rotations = model_loaded.predict(dlatest_new)
    df_latest_new['Pred_RemainingRotations'] = pred_remaining_rotations
    
    # Convert to days and dates
    global_avg_rph = df_new[df_new['Rotations_LastHour'] > 0]['Rotations_LastHour'].mean()
    df_latest_new['RotationsPerHour_Estimate'] = df_latest_new['Rotations_LastHour'].replace(0, global_avg_rph)
    df_latest_new['Pred_DaysLeft'] = (df_latest_new['Pred_RemainingRotations'] / df_latest_new['RotationsPerHour_Estimate']) / 24.0
    df_latest_new['Pred_DaysLeft'] = df_latest_new['Pred_DaysLeft'].clip(0, 365)
    
    current_time = pd.Timestamp.now()
    df_latest_new['Pred_ReplacementDate'] = current_time + pd.to_timedelta(df_latest_new['Pred_DaysLeft'], unit='D')
    df_latest_new['PredictionTimestamp'] = current_time
    
    # Risk levels
    def assign_risk_level(days):
        if days <= (1/24):
            return '🔴 CRITICAL'
        elif days <= 1:
            return '🟠 HIGH'
        elif days <= 7:
            return '🟡 MEDIUM'
        else:
            return '🟢 LOW'
    
    df_latest_new['RiskLevel'] = df_latest_new['Pred_DaysLeft'].apply(assign_risk_level)
    df_latest_new['IsCritical'] = (df_latest_new['Pred_DaysLeft'] <= (1/24)).astype(int)
    
    print(f"✅ Generated {len(pred_remaining_rotations)} predictions")
    print(f"🚨 Critical machines: {df_latest_new['IsCritical'].sum()}")
    
    # Step 5: Save predictions (append mode)
    prediction_output_cols = [
        'PredictionTimestamp', 'Timestamp', 'MachineID', 'CumulativeBitRotation',
        'rotation_in_cycle', 'Rotations_LastHour', 'RotationsPerHour_Estimate',
        'Pred_RemainingRotations', 'Pred_DaysLeft', 'Pred_ReplacementDate',
        'RiskLevel', 'IsCritical'
    ]
    
    df_predictions_new = df_latest_new[prediction_output_cols].copy()
    df_predictions_new['ModelVersion'] = 'v1_xgboost'
    
    spark_predictions_new = spark.createDataFrame(df_predictions_new)
    spark_predictions_new.write.mode("append").saveAsTable("MaintenanceML.replacement_predictions")
    
    print(f"✅ Predictions saved to Lakehouse")
    print(f"✅ Daily prediction update completed: {datetime.now()}")

# Uncomment to run daily update
# daily_prediction_update()

## Part 12: Power BI Integration

**Connect Power BI to Lakehouse:**

1. Open Power BI Desktop
2. Get Data → Microsoft Fabric → Lakehouse
3. Select workspace → `MaintenanceML` lakehouse
4. Load table: `replacement_predictions`

**DAX Measures:**

```dax
// Average Days Until Replacement
AvgDaysLeft = AVERAGE(replacement_predictions[Pred_DaysLeft])

// Critical Machines Count
CriticalMachines = 
CALCULATE(
    DISTINCTCOUNT(replacement_predictions[MachineID]),
    replacement_predictions[RiskLevel] = "🔴 CRITICAL"
)

// High Risk Machines Count
HighRiskMachines = 
CALCULATE(
    DISTINCTCOUNT(replacement_predictions[MachineID]),
    replacement_predictions[RiskLevel] IN {"🔴 CRITICAL", "🟠 HIGH"}
)

// Next Replacement Date
NextReplacementDate = 
CALCULATE(
    MIN(replacement_predictions[Pred_ReplacementDate]),
    replacement_predictions[IsCritical] = 1
)

// Risk Level Dynamic Color
RiskColor = 
SWITCH(
    SELECTEDVALUE(replacement_predictions[RiskLevel]),
    "🔴 CRITICAL", "#DC3545",  // Red
    "🟠 HIGH", "#FD7E14",      // Orange
    "🟡 MEDIUM", "#FFC107",    // Yellow
    "🟢 LOW", "#28A745",       // Green
    "#6C757D"                  // Gray default
)
```

**Recommended Visuals:**

### 1. **KPI Cards (New Card Visual)**
Displays key metrics at a glance with large numbers and optional trend indicators.

**How to Create:**
1. In Power BI Desktop, select **Insert** → **New visual** → **(new) Card visual** icon
2. From **Data pane**, drag these measures to the card:
   - **Total Machines**: `DISTINCTCOUNT(replacement_predictions[MachineID])`
   - **Critical Machines**: `CriticalMachines` (use DAX from Part 12)
   - **Avg Days Left**: `AvgDaysLeft` (use DAX from Part 12)
   - **Next Replacement**: `NextReplacementDate` (use DAX from Part 12)
3. **Format** each card:
   - **Visual** → **Callout value** → Set font size (48-72pt for large displays)
   - **Visual** → **Category label** → Enable and customize title (e.g., "Critical Machines")
   - **Visual** → **Effects** → Add background color based on severity (red for critical)
4. Arrange 4 cards horizontally at top of dashboard

**Pro Tip:** Group multiple cards in a container for unified styling and easier management.

---

### 2. **Risk Matrix Table (Table with Conditional Formatting)**
Shows all machines with color-coded risk levels for quick scanning.

**How to Create:**
1. Select **Table** visual from Visualizations pane
2. Add these columns in order:
   - `MachineID`
   - `Pred_DaysLeft` (rename to "Days Until Replacement")
   - `Pred_ReplacementDate` (rename to "Predicted Date")
   - `Pred_RemainingRotations` (rename to "Remaining Rotations")
   - `RiskLevel`
3. **Apply Conditional Formatting:**
   - Right-click `RiskLevel` column → **Conditional formatting** → **Background color**
   - **Format style**: Rules
   - Create 4 rules:
     * "🔴 CRITICAL" → Red (#DC3545)
     * "🟠 HIGH" → Orange (#FD7E14)
     * "🟡 MEDIUM" → Yellow (#FFC107)
     * "🟢 LOW" → Green (#28A745)
4. **Format Days Left Column:**
   - Right-click `Pred_DaysLeft` → **Conditional formatting** → **Data bars**
   - Set minimum (0) and maximum (30) for scale
   - Choose gradient color (red to green)
5. **Additional Formatting:**
   - **Visual** → **Style** → Choose "Minimal" or "Bold header"
   - **Visual** → **Grid** → Enable row padding for readability
   - **Visual** → **Values** → Set text size (11-12pt)
   - **General** → **Size** → Set appropriate height to show all machines

**Interaction:** Users can click rows to filter other visuals on the page.

---

### 3. **Replacement Timeline (Gantt Chart)**
Visual timeline showing when each machine needs replacement, color-coded by risk.

**How to Create:**
1. **Download Official Microsoft Gantt Chart:**
   - Go to **AppSource** (Home → Get Data → More → Get visuals from AppSource)
   - Search for "Gantt" and install **"Gantt" by Microsoft** (free, official visual)
   - Repository: https://github.com/microsoft/powerbi-visuals-gantt
2. Add **Gantt** visual to canvas
3. **Configure Data Fields:**
   - **Task**: `MachineID` (required - identifies each machine/task)
   - **Start Date**: `PredictionTimestamp` (required - when prediction was made/current date)
   - **End Date**: `Pred_ReplacementDate` (optional - if provided, overrides duration)
   - **Duration**: `Pred_DaysLeft` (optional - used if End Date not provided)
   - **Completion**: Create calculated column = `0` (predictive, not started yet)
     * Format as decimal (0.0 = 0%, 0.85 = 85%)
   - **Resource**: `RiskLevel` (optional - shows resource assignment in task bars)
   - **Legend**: `RiskLevel` (optional - for color-coding by risk category)
4. **Format Settings:**
   - **General Settings:**
     * Enable **Group tasks** to collapse/expand by hierarchy
     * Enable **Scroll to current time** to auto-focus on today's date
   - **Legend Settings:**
     * Position: Top or Right
     * Show legend to display risk level colors
   - **Task Config:**
     * **Height**: 40-50 pixels for readability
     * **Color**: Set default or use legend colors
   - **Task Labels:**
     * **Show**: Enabled
     * **Width**: 150-200 pixels
     * **Font size**: 11-12pt
     * Display `MachineID` on left axis
   - **Task Resource (shows in bars):**
     * **Show**: Enabled to display risk level inside/outside bars
     * **Position**: Inside, Right, or Top
     * **Font size**: 10-11pt
   - **Date Type:**
     * Select: Day, Week, or Month depending on timeline scope
     * For 30-60 day predictions, use "Day" or "Week"
   - **Today Line:**
     * **Enable**: Shows vertical red line at current date for reference
5. **Color Coding by Risk:**
   - In **Legend** settings → **Data colors**
   - Manually set colors for each risk level:
     * 🔴 CRITICAL: #DC3545 (Red)
     * 🟠 HIGH: #FD7E14 (Orange)  
     * 🟡 MEDIUM: #FFC107 (Yellow)
     * 🟢 LOW: #28A745 (Green)
6. **Timeline Features:**
   - Tasks automatically rendered as horizontal bars from Start to End Date
   - Bar length represents duration (days until replacement)
   - X-axis shows calendar dates with automatic scaling
   - Y-axis shows machine names (from Task field)
   - Hover over bars to see tooltip with details

**Pro Tips:**
- Sort tasks by `Pred_ReplacementDate` (ascending) to show urgent machines at top
- Add **Parent** field if you have machine groups/locations for hierarchical view
- Use **Milestones** field to mark actual replacement events
- Enable **Days Off** settings to gray out weekends/holidays if relevant

**Alternative (Built-in Stacked Bar Chart):**
If custom visual installation is restricted:
- Use **Stacked Bar Chart**
- **Axis**: `MachineID`
- **Values**: `Pred_DaysLeft`
- **Legend**: `RiskLevel`
- Rotate to horizontal for timeline effect
- Less accurate but still shows relative urgency

---

### 4. **Risk Distribution (Donut Chart)**
Shows proportion of machines in each risk category.

**How to Create:**
1. Select **Donut chart** icon from Visualizations pane
2. **Configure fields:**
   - **Legend**: `RiskLevel`
   - **Values**: `DISTINCTCOUNT(MachineID)` or just drag `MachineID` to Values
3. **Format for clarity:**
   - **Visual** → **Legend** → Position: "Right" or "Bottom"
   - **Visual** → **Detail labels**:
     * Enable labels
     * Display: "Category, percentage" or "Category, value"
     * Label position: "Outside"
     * Font size: 11-12pt
   - **Visual** → **Slices**:
     * Manually set colors to match risk levels:
       - 🔴 CRITICAL: #DC3545 (Red)
       - 🟠 HIGH: #FD7E14 (Orange)
       - 🟡 MEDIUM: #FFC107 (Yellow)
       - 🟢 LOW: #28A745 (Green)
     * Sort by: Custom order (CRITICAL → HIGH → MEDIUM → LOW)
   - **General** → **Title**: "Machine Risk Distribution"
   - **General** → **Effects** → Add subtle shadow
4. **Add tooltip:**
   - **Visual** → **Tooltip** → Enable
   - Add fields: `MachineID`, `Pred_DaysLeft`, `Pred_ReplacementDate`

**Interaction:** Clicking a slice filters other visuals to show only machines in that risk category.

---

### 5. **Prediction Accuracy Over Time (Line Chart)**
Tracks model performance to detect degradation and trigger retraining.

**How to Create:**
1. Select **Line chart** from Visualizations pane
2. **Configure fields:**
   - **X-axis**: `PredictionTimestamp` (convert to Date only, not datetime)
     * Right-click → Choose "PredictionTimestamp" instead of "Date Hierarchy"
   - **Y-axis**: Create measure for rolling MAE:
     ```dax
     Rolling_MAE = 
     CALCULATE(
         AVERAGEX(
             replacement_predictions,
             ABS([Pred_DaysLeft] - [RemainingDays_Actual])
         ),
         DATESINPERIOD(
             'Date'[Date],
             LASTDATE('Date'[Date]),
             -7,
             DAY
         )
     )
     ```
   - **Legend** (optional): `ModelVersion` to compare different model versions
3. **Format chart:**
   - **Visual** → **Lines**:
     * Stroke width: 3px
     * Line style: Solid
     * Add markers: Enabled (circle, 8px)
   - **Visual** → **Reference line** (Y-axis):
     * Add horizontal line for "Acceptable MAE Threshold" (e.g., 2.0 days)
     * Color: Red dashed line
     * Label: "Retrain Threshold"
   - **Visual** → **X-axis**:
     * Type: Continuous
     * Display units: Auto
     * Title: "Prediction Date"
   - **Visual** → **Y-axis**:
     * Title: "Mean Absolute Error (Days)"
     * Start: 0
     * Display units: None
   - **Visual** → **Data labels**: Disabled (too cluttered)
   - **General** → **Title**: "Model Performance: MAE Trend (7-Day Rolling Average)"
4. **Add drill-through:**
   - Create detail page showing individual predictions
   - Enable drill-through on this chart to see which machines had high errors

**Pro Tip:** Add a **Card visual** next to line chart showing "Current MAE" vs "Last Week MAE" to quickly spot changes.

**Alternative Metrics:**
- Also plot R² score over time (higher is better, target >0.7)
- Show count of predictions per day (monitoring data ingestion)
- Display % of critical alerts that resulted in actual replacements (alert accuracy)

---

### 6. **BONUS: Next Replacements Timeline (Table Visual)**
Shows upcoming replacements in chronological order with countdown timer.

**How to Create:**
1. Select **Table** visual
2. Add columns:
   - `Pred_ReplacementDate` (sorted ascending)
   - `MachineID`
   - `Pred_DaysLeft`
   - Create calculated column:
     ```dax
     Days_From_Now = DATEDIFF(TODAY(), [Pred_ReplacementDate], DAY)
     Countdown = 
     IF([Days_From_Now] < 0, 
         "OVERDUE by " & ABS([Days_From_Now]) & " days", 
         [Days_From_Now] & " days"
     )
     ```
   - `RiskLevel`
3. **Filter visual:** Only show next 30 days
   - **Filters** → `Pred_ReplacementDate` → "is in the next 30 days"
4. **Conditional format** `Countdown` column:
   - Background color based on days (red < 1, orange < 7, yellow < 14)

---

### **Dashboard Layout Best Practices:**

**Page 1: Executive Dashboard**
```
┌─────────────────────────────────────────────────────┐
│ [Total Machines] [Critical] [Avg Days] [Next Date] │ ← KPI Cards
├──────────────────────┬──────────────────────────────┤
│                      │                              │
│  Risk Distribution   │   Replacement Timeline       │ ← Donut + Gantt
│    (Donut Chart)     │      (Gantt Chart)           │
│                      │                              │
├──────────────────────┴──────────────────────────────┤
│          Risk Matrix Table (All Machines)           │ ← Table
└─────────────────────────────────────────────────────┘
```

**Page 2: Model Performance**
```
┌─────────────────────────────────────────────────────┐
│  [Current MAE]  [Last Week MAE]  [Predictions/Day]  │
├─────────────────────────────────────────────────────┤
│                                                     │
│        Prediction Accuracy Over Time                │ ← Line Chart
│              (7-Day Rolling MAE)                    │
│                                                     │
├─────────────────────────────────────────────────────┤
│     Actual vs Predicted Scatter + Error Dist.       │
└─────────────────────────────────────────────────────┘
```

---

### **Final Setup Checklist:**

✅ **Enable Auto-Refresh:** Settings → Dataset → Schedule refresh (daily at 2 AM)  
✅ **Add Slicers:** MachineID, Date Range, Risk Level for filtering  
✅ **Enable Drill-Through:** From summary to detail pages  
✅ **Mobile Layout:** Create responsive layout for phone access  
✅ **Set Alerts:** Data alerts on Critical Machines count (>5 triggers email)  
✅ **Add Bookmarks:** Save views like "Critical Only" or "This Week"  
✅ **Publish to Workspace:** Share with maintenance team with edit permissions  
✅ **Pin to Teams:** Add dashboard tab in maintenance Teams channel

## Part 13: Alerting and Work Order Automation via Email

**Prerequisites:**
- Access to Power Automate (included with Power BI Pro or Premium license)
- Office 365 Outlook or Outlook.com email account
- Predictions saved to Fabric Lakehouse table: `MaintenanceML.replacement_predictions`

---

### **Step-by-Step: Create Power Automate Flow for Email Alerts**

#### **Step 1: Navigate to Power Automate**
1. Go to **Power Automate**: https://make.powerautomate.com
2. Sign in with your Microsoft 365 account (same account as Power BI/Fabric)
3. Ensure you're in the **correct environment** (same as your Fabric workspace)
   - Check environment dropdown in top-right corner

---

#### **Step 2: Create Automated Cloud Flow**
1. On the left navigation pane, select **Create**
2. Under **Start from blank**, select **Automated cloud flow**
3. In the dialog box:
   - **Flow name**: Enter `Critical Machine Replacement Alert`
   - **Choose your flow's trigger**: Search for **Dataverse**
   - Select **When a row is added, modified or deleted** (Microsoft Dataverse connector)
4. Click **Create**

![Screenshot example: Automated cloud flow creation dialog]

---

#### **Step 3: Configure Dataverse Trigger**

**Configure the trigger to monitor new predictions:**

1. **Change type**: Select **Added** (only trigger when new predictions are written)
   
2. **Table name**: 
   - Click the dropdown
   - Search for your Lakehouse table: Type `replacement_predictions`
   - Select: `MaintenanceML.replacement_predictions`
   - *(Note: Fabric Lakehouse tables appear as Dataverse tables in OneLake)*

3. **Scope**: Select **Organization** (monitor all rows in the environment)

4. **Show advanced options** (click to expand):
   - **Select columns**: Leave blank (we'll filter in next step)
   - **Filter rows**: Enter the following OData filter expression:
     ```
     IsCritical eq 1
     ```
     This ensures the flow ONLY triggers for critical machines (< 1 hour remaining)

5. **Run as**: Select **Modifying user** (optional, for audit trail)

**Your trigger should look like:**
```
When a row is added, modified or deleted
├─ Change type: Added
├─ Table name: MaintenanceML.replacement_predictions
├─ Scope: Organization
├─ Filter rows: IsCritical eq 1
└─ Run as: Modifying user
```

---

#### **Step 4: Add Condition (Optional - Double Check)**

*This step is optional since we already filtered in the trigger, but adds redundancy:*

1. Click **+ New step**
2. Search for **Condition** (Control)
3. Configure condition:
   - **Choose a value**: Click inside box → Select **IsCritical** from Dynamic content
   - **Operator**: Select `is equal to`
   - **Choose a value**: Enter `1`

This creates two branches: **If yes** (critical) and **If no** (not critical).

**Continue building actions inside "If yes" branch:**

---

#### **Step 5: Send Email Alert to Maintenance Team**

1. Inside the **If yes** branch (or directly after trigger if you skipped Step 4), click **Add an action**
2. Search for **send email**
3. Select **Send an email (V2)** - Office 365 Outlook
   - If using personal Microsoft account, select **Outlook.com** connector instead

4. **Sign in** if prompted to create connection to your Outlook account

5. **Configure Email Action:**

   **A. To (Recipients):**
   - Enter email addresses separated by semicolons:
     ```
     maintenance-team@yourcompany.com; supervisor@yourcompany.com
     ```
   - Or click **Dynamic content** → Use email field from a table if stored

   **B. Subject:**
   - Enter static text and add dynamic content:
     ```
     🔴 CRITICAL: Machine Replacement Required - [MachineID]
     ```
   - To add dynamic content:
     1. Click inside Subject field
     2. Click **Lightning bolt icon** (⚡) to open Dynamic content panel
     3. Under **When a row is added, modified or deleted**, select:
        - **MachineID** (inserts after "Machine")
   
   **C. Body (Email Message):**
   - Click inside Body field
   - Click **Lightning bolt icon** (⚡) to open Dynamic content
   - Compose message with dynamic fields:

   **Example email body template:**
   ```
   🚨 URGENT: Critical Machine Replacement Alert
   
   A machine requires IMMEDIATE attention:
   
   📍 Machine Details:
   - Machine ID: [MachineID]
   - Current Rotations: [CumulativeBitRotation]
   - Remaining Rotations: [Pred_RemainingRotations]
   
   ⏰ Timeline:
   - Days Left: [Pred_DaysLeft] days ([Pred_DaysLeft]*24 hours)
   - Predicted Replacement Date: [Pred_ReplacementDate]
   - Alert Generated: [PredictionTimestamp]
   
   🔧 Recommended Action:
   Schedule immediate maintenance to prevent unexpected downtime.
   
   📊 Activity Metrics:
   - Rotations Last Hour: [Rotations_LastHour]
   - Average Torque: [AvgTorque_LastHour] Nm
   
   Risk Level: [RiskLevel]
   
   ---
   This alert was generated automatically by the Predictive Maintenance ML Model.
   View full dashboard: [Your Power BI Dashboard Link]
   ```

   **How to insert dynamic content:**
   - Click where you want to insert data (e.g., after "Machine ID:")
   - Click **Lightning bolt** icon
   - Search for field name (e.g., `MachineID`)
   - Click the field to insert it

   **Dynamic fields available:**
   - MachineID
   - CumulativeBitRotation
   - Pred_RemainingRotations
   - Pred_DaysLeft
   - Pred_ReplacementDate
   - PredictionTimestamp
   - Rotations_LastHour
   - AvgTorque_LastHour
   - RiskLevel

   **D. Advanced Options (Optional):**
   - Click **Show advanced options**
   - **Importance**: Select **High** (adds red exclamation mark in Outlook)
   - **Attachments**: Leave blank (or add Power BI report snapshot if configured)

---

#### **Step 6: Send Work Order Email (Optional Second Action)**

To create a work order via email (e.g., to a ticketing system like ServiceNow, Jira Service Desk):

1. Click **Add an action** (below the first email action)
2. Search for **send email**
3. Select **Send an email (V2)** again
4. Configure:
   - **To**: `workorders@yourcompany.com` or ticketing system email
   - **Subject**: 
     ```
     [WORK ORDER] Preventive Maintenance - Machine [MachineID]
     ```
   - **Body**:
     ```
     Work Order Request
     
     Type: Preventive Maintenance - Bit Replacement
     Asset: [MachineID]
     Priority: High
     Scheduled Date: [Pred_ReplacementDate]
     
     Description:
     Predicted bit replacement required based on ML analysis.
     Current cumulative rotations: [CumulativeBitRotation]
     Estimated remaining useful life: [Pred_DaysLeft] days
     
     Action Required:
     Replace drill bit before [Pred_ReplacementDate] to prevent unplanned downtime.
     
     Auto-generated by Power Automate - Predictive Maintenance System
     ```

---

#### **Step 7: Add Teams Notification (Optional Third Action)**

1. Click **Add an action**
2. Search for **Teams**
3. Select **Post message in a chat or channel** (Microsoft Teams)
4. Sign in to Teams if prompted
5. Configure:
   - **Post as**: Flow bot
   - **Post in**: Channel
   - **Team**: Select your team (e.g., "Maintenance Operations")
   - **Channel**: Select channel (e.g., "Alerts" or "General")
   - **Message**:
     ```
     🔴 **CRITICAL MACHINE ALERT**
     
     Machine **[MachineID]** needs replacement in **[Pred_DaysLeft]** days!
     
     Predicted Date: [Pred_ReplacementDate]
     Risk Level: [RiskLevel]
     
     📧 Alert email sent to maintenance team.
     ```

---

#### **Step 8: Save and Test the Flow**

1. **Name your flow** (top-left): `Critical Machine Replacement Alert`
2. Click **Save** (top-right)
3. **Test the flow**:
   - Click **Test** button (top-right)
   - Select **Manually**
   - Click **Test** → **Run flow**
   - Manually add a test record to your Lakehouse table with `IsCritical = 1`
   - OR wait for next ML prediction run to trigger automatically

4. **Check Flow Run History**:
   - Go back to **My flows**
   - Select your flow
   - View **28-day run history**
   - Click on a run to see execution details (success/failure/which actions ran)

---

#### **Step 9: Enable and Monitor**

1. Ensure flow is **Turned On** (toggle in top-right of flow details page)
2. **Monitor flow runs**:
   - Power Automate → My flows → Your flow → Run history
   - Check for errors and review execution time
3. **Set up email notifications for flow failures**:
   - Flow settings → Notifications → Enable "Email notification on flow failure"

---

### **🎯 Flow Architecture Summary:**

```
Trigger: When row added to MaintenanceML.replacement_predictions
   └─ Filter: IsCritical eq 1
      └─ Condition: IsCritical = 1 (optional redundancy)
         └─ If YES:
            ├─ Send Email Alert → Maintenance Team
            ├─ Send Work Order Email → Ticketing System (optional)
            └─ Post Teams Message → Maintenance Channel (optional)
```

---

### **📧 Example: Final Email Preview**

**Subject:** 🔴 CRITICAL: Machine Replacement Required - MACHINE-042

**Body:**
```
🚨 URGENT: Critical Machine Replacement Alert

A machine requires IMMEDIATE attention:

📍 Machine Details:
- Machine ID: MACHINE-042
- Current Rotations: 1,245,680
- Remaining Rotations: 320

⏰ Timeline:
- Days Left: 0.5 days (12 hours)
- Predicted Replacement Date: 2025-11-19 14:30:00
- Alert Generated: 2025-11-19 02:30:00

🔧 Recommended Action:
Schedule immediate maintenance to prevent unexpected downtime.

📊 Activity Metrics:
- Rotations Last Hour: 26.7
- Average Torque: 4.2 Nm

Risk Level: 🔴 CRITICAL

---
This alert was generated automatically by the Predictive Maintenance ML Model.
View full dashboard: https://app.powerbi.com/groups/.../dashboards/...
```

---

### **Alternative: Email Alert (Python)**

In [ ]:
# Example: Send email alert for critical machines (requires email configuration)
# Uncomment and configure SMTP settings to enable

def send_critical_alerts(df_predictions):
    """
    Send email alerts for critical machines
    """
    critical_machines = df_predictions[df_predictions['IsCritical'] == 1]
    
    if len(critical_machines) > 0:
        # Compose alert message
        message = f"🔴 CRITICAL ALERT: {len(critical_machines)} machines need immediate attention\n\n"
        
        for _, row in critical_machines.iterrows():
            message += f"Machine {row['MachineID']}:\n"
            message += f"  Remaining: {row['Pred_DaysLeft']:.2f} days ({row['Pred_DaysLeft']*24:.1f} hours)\n"
            message += f"  Predicted Replacement: {row['Pred_ReplacementDate']}\n"
            message += f"  Current Rotations: {row['CumulativeBitRotation']:,.0f}\n\n"
        
        print(f"📧 Alert Message:\n{message}")
        
        # TODO: Add SMTP email sending code
        # import smtplib
        # ...
    else:
        print("✅ No critical alerts")

# Run alert check
send_critical_alerts(df_predictions)

## Part 14: Build Work Order Management System with Power Apps

**Overview:**

Create a canvas app that allows maintenance engineers to:
1. View work orders automatically created from Power Automate alerts
2. Update work order status (New → In Progress → Completed)
3. Add notes and actual replacement timestamps
4. Track maintenance history per machine

This integrates with Part 13's Power Automate flow to create a complete ticketing system.

---

### **Choose Your Backend: Dataverse vs SharePoint List**

You have two options for storing work order data:

| Feature | **Option A: Dataverse** | **Option B: SharePoint List** |
|---------|------------------------|------------------------------|
| **Cost** | Requires Power Apps premium license | Included with Microsoft 365 |
| **Complexity** | More setup steps | Simpler, faster setup |
| **Features** | Advanced relationships, business rules | Basic list functionality |
| **Scalability** | Better for high volume (>100K records) | Good for moderate volume (<50K) |
| **Integration** | Native Power Platform integration | Good Office 365 integration |
| **Best For** | Enterprise solutions, complex workflows | Departmental apps, quick deployments |

**Recommendation:**
- **Start with SharePoint List** if you want quick deployment and already use Microsoft 365
- **Use Dataverse** if you need advanced features, relationships, or plan to scale significantly

---

## **OPTION A: Using Dataverse Backend**

### **Step 1A: Create Work Order Table in Dataverse**

**1.1. Navigate to Power Apps Portal**
1. Go to https://make.powerapps.com
2. Select your environment (same as Fabric workspace)
3. On the left navigation, select **Tables** (under Dataverse)

**1.2. Create New Table**
1. Click **+ New table** → **Add columns and data**
2. **Table name**: `Work Orders`
3. **Primary column name**: `Work Order ID` (auto-generated)

**1.3. Add Columns to Work Order Table**

Click **+ (Add column)** and create the following columns:

| Column Name | Data Type | Required | Description |
|------------|-----------|----------|-------------|
| **MachineID** | Text | Yes | Machine identifier |
| **WorkOrderType** | Choice | Yes | Options: Preventive Maintenance, Corrective Maintenance, Inspection |
| **Priority** | Choice | Yes | Options: Critical, High, Medium, Low |
| **Status** | Choice | Yes | Options: New, Assigned, In Progress, Completed, Cancelled |
| **PredictedReplacementDate** | Date and Time | Yes | From ML prediction |
| **DaysRemaining** | Decimal | Yes | Days left until replacement |
| **RemainingRotations** | Whole Number | No | Rotations left |
| **CurrentRotations** | Whole Number | No | Current cumulative rotations |
| **AssignedTo** | Lookup (User) | No | Engineer assigned to work order |
| **CreatedByFlow** | Yes/No | No | True if auto-generated |
| **ActualCompletionDate** | Date and Time | No | Actual date work completed |
| **Notes** | Text (Multi-line) | No | Engineer notes |
| **AlertTimestamp** | Date and Time | No | When alert was generated |

**Choice Field Details:**

**WorkOrderType:**
- Preventive Maintenance (Default)
- Corrective Maintenance
- Inspection

**Priority:**
- Critical (Default for IsCritical=1)
- High
- Medium
- Low

**Status:**
- New (Default)
- Assigned
- In Progress
- Completed
- Cancelled

**1.4. Save Table**
1. Click **Save** (bottom-right)
2. Click **Create** to finalize table

**Note:** The table logical name will be `cr123_workorders` (with your custom prefix)

---

### **Step 2A: Update Power Automate Flow to Create Work Orders (Dataverse)**

Go back to your **Critical Machine Replacement Alert** flow from Part 13 and add work order creation.

**2.1. Add New Action After Email Alert**

1. In your Power Automate flow, after the **Send an email** action, click **+ New step**
2. Search for **Dataverse**
3. Select **Add a new row** (Microsoft Dataverse)

**2.2. Configure Add a New Row Action**

1. **Table name**: Select `Work Orders`
2. **Show advanced options**: Click to expand

**Fill in the following fields:**

| Field | Value (Use Dynamic Content) |
|-------|----------------------------|
| **MachineID** | MachineID |
| **Work Order Type** | Preventive Maintenance |
| **Priority** | Critical |
| **Status** | New |
| **Predicted Replacement Date** | Pred_ReplacementDate |
| **Days Remaining** | Pred_DaysLeft |
| **Remaining Rotations** | Pred_RemainingRotations |
| **Current Rotations** | CumulativeBitRotation |
| **Created By Flow** | Yes |
| **Alert Timestamp** | PredictionTimestamp |
| **Notes** | Enter text: `Auto-generated work order from ML prediction. Risk Level: [RiskLevel]` |

3. **Save** the flow

**Flow Architecture Now:**

```
Trigger: When row added to replacement_predictions (IsCritical eq 1)
  └─ Send Email Alert → Maintenance Team
  └─ Add a new row → Work Orders table (Dataverse) ✨ NEW
  └─ Post Teams Message (optional)
```

---

### **Step 3A: Create Power Apps Canvas App (Dataverse)**

**3.1. Create App from Data**

1. Go to https://make.powerapps.com
2. On Home screen, select **Start with data** → **Select an existing table**
3. Search for and select **Work Orders** table
4. Click **Create app**

Power Apps generates a three-screen app automatically:
- **BrowseScreen1**: List all work orders
- **DetailScreen1**: View work order details
- **EditScreen1**: Edit/create work orders

**3.2. Customize Browse Screen (Work Order List)**

1. Select **BrowseScreen1** from Tree View
2. Select **BrowseGallery1** (the gallery showing work orders)

**Customize Gallery Layout:**

1. In Properties pane → **Layout**: Select **Title, subtitle, and body**
2. Click **Edit** next to Fields
3. Configure fields:
   - **Title**: `Work Order ID` (or `Name`)
   - **Subtitle**: `MachineID`
   - **Body**: `Status`
4. Add additional labels for Priority and Days Remaining:
   - Insert **Label** control inside gallery template (first item)
   - **Text**: `"Priority: " & ThisItem.Priority`
   - **Color**: Use conditional formatting:
     ```power
     If(ThisItem.Priority = 'Priority (Work Orders)'.Critical, 
        ColorValue("#DC3545"), 
        If(ThisItem.Priority = 'Priority (Work Orders)'.High, 
           ColorValue("#FD7E14"), 
           ColorValue("#6C757D")
        )
     )
     ```

**Add Status Badge:**
- Insert **Label** control
- **Text**: `ThisItem.Status`
- **Fill** (background color):
  ```power
  Switch(ThisItem.Status,
      'Status (Work Orders)'.New, ColorValue("#007BFF"),
      'Status (Work Orders)'.'In Progress', ColorValue("#FFC107"),
      'Status (Work Orders)'.Completed, ColorValue("#28A745"),
      ColorValue("#6C757D")
  )
  ```
- **Color**: White

**Add Filter/Sort:**

1. Select **Search bar** (TextSearchBox1)
2. Keep default search functionality
3. Add **Dropdown** for status filter:
   - Insert → Input → **Dropdown**
   - **Items**: 
     ```power
     ["All", "New", "In Progress", "Completed"]
     ```
   - Position at top of screen
   - **Default**: "All"

4. Update Gallery **Items** property:
   ```power
   SortByColumns(
       If(Dropdown1.Selected.Value = "All",
          Search('Work Orders', TextSearchBox1.Text, "MachineID"),
          Filter(
              Search('Work Orders', TextSearchBox1.Text, "MachineID"),
              Status = Dropdown1.Selected.Value
          )
       ),
       "Priority", Descending,
       "Alert Timestamp", Descending
   )
   ```

**3.3. Customize Detail Screen (View Work Order)**

1. Select **DetailScreen1** from Tree View
2. Select **DetailForm1** (the display form)

**Add/Reorder Fields:**

1. In Properties pane → **Edit fields**
2. Click **+ Add field** and add all important fields:
   - Work Order ID
   - MachineID
   - Work Order Type
   - Priority
   - Status
   - Days Remaining
   - Predicted Replacement Date
   - Current Rotations
   - Remaining Rotations
   - Assigned To
   - Alert Timestamp
   - Actual Completion Date
   - Notes
3. Drag to reorder as listed above
4. Click outside to close

**Add Conditional Formatting for Priority:**

1. Select **Priority data card** in form
2. Select the label showing priority value
3. **Color** property:
   ```power
   If(Parent.Default = 'Priority (Work Orders)'.Critical, 
      ColorValue("#DC3545"), 
      ColorValue("#000000")
   )
   ```

**3.4. Customize Edit Screen (Update Work Order)**

1. Select **EditScreen1** from Tree View
2. Select **EditForm1** (the edit form)

**Configure Form Mode:**

Set **OnVisible** property of EditScreen1:
```power
If(
    IsBlank(BrowseGallery1.Selected),
    NewForm(EditForm1),
    EditForm(EditForm1)
)
```

**Make Certain Fields Read-Only:**

For auto-generated fields, make them read-only:
1. Select data card for **MachineID**
2. In Advanced pane → **DisplayMode**: `DisplayMode.View`
3. Repeat for:
   - Work Order ID
   - Predicted Replacement Date
   - Days Remaining
   - Current Rotations
   - Created By Flow
   - Alert Timestamp

**Add Engineer Assignment:**

1. Ensure **Assigned To** field is in form (user lookup)
2. Keep it editable so engineers can assign themselves

**Status Update Dropdown:**

1. Select **Status** data card
2. Control type should be **Dropdown** (default for Choice fields)
3. Keep editable for status updates

**3.5. Add Action Buttons**

**A. "Assign to Me" Button (on DetailScreen1):**

1. Insert **Button** control
2. **Text**: "Assign to Me"
3. **OnSelect**:
   ```power
   Patch('Work Orders',
       BrowseGallery1.Selected,
       {'Assigned To': User()}
   );
   Refresh('Work Orders');
   ```
4. **DisplayMode**: 
   ```power
   If(IsBlank(DetailForm1.LastSubmit.'Assigned To'), 
      DisplayMode.Edit, 
      DisplayMode.Disabled
   )
   ```

**B. "Start Work" Button (on DetailScreen1):**

1. Insert **Button** control
2. **Text**: "Start Work"
3. **OnSelect**:
   ```power
   Patch('Work Orders',
       BrowseGallery1.Selected,
       {
           Status: 'Status (Work Orders)'.'In Progress',
           'Assigned To': User()
       }
   );
   Refresh('Work Orders');
   Back();
   ```
4. **Visible**: 
   ```power
   DetailForm1.LastSubmit.Status = 'Status (Work Orders)'.New ||
   DetailForm1.LastSubmit.Status = 'Status (Work Orders)'.Assigned
   ```

**C. "Complete Work Order" Button (on EditScreen1):**

1. Insert **Button** control
2. **Text**: "Mark as Completed"
3. **OnSelect**:
   ```power
   Patch('Work Orders',
       BrowseGallery1.Selected,
       {
           Status: 'Status (Work Orders)'.Completed,
           'Actual Completion Date': Now()
       }
   );
   SubmitForm(EditForm1);
   Back();
   ```
4. **Visible**: 
   ```power
   EditForm1.Mode = FormMode.Edit &&
   (ThisItem.Status = 'Status (Work Orders)'.'In Progress' ||
    ThisItem.Status = 'Status (Work Orders)'.Assigned)
   ```

**3.6. Save and Publish App**

1. **File** → **Save**
2. **App name**: `Maintenance Work Orders`
3. Click **Save**
4. Click **Publish** → **Publish this version**

---

## **OPTION B: Using SharePoint List Backend**

### **Step 1B: Create Work Orders SharePoint List**

**1.1. Navigate to SharePoint Site**
1. Go to your SharePoint site (e.g., `https://yourtenant.sharepoint.com/sites/maintenance`)
2. If you don't have a site, create one:
   - Go to SharePoint home → **+ Create site** → **Team site**
   - Name: `Maintenance Operations`
3. On the site home page, click **+ New** → **List**

**1.2. Create New List**
1. Select **Blank list**
2. **Name**: `Work Orders`
3. **Description**: `Predictive maintenance work order tracking`
4. Click **Create**

**1.3. Add Columns to SharePoint List**

Click **+ Add column** and create the following columns:

| Column Name | Column Type | Required | Options/Settings |
|------------|-------------|----------|------------------|
| **MachineID** | Single line of text | Yes | Max length: 50 |
| **WorkOrderType** | Choice | Yes | Choices: Preventive Maintenance, Corrective Maintenance, Inspection (Default: Preventive Maintenance) |
| **Priority** | Choice | Yes | Choices: Critical, High, Medium, Low (Default: Critical, use color coding) |
| **Status** | Choice | Yes | Choices: New, Assigned, In Progress, Completed, Cancelled (Default: New) |
| **PredictedReplacementDate** | Date and time | Yes | Include time: Yes |
| **DaysRemaining** | Number | Yes | Type: Decimal |
| **RemainingRotations** | Number | No | Type: Number (no decimals) |
| **CurrentRotations** | Number | No | Type: Number (no decimals) |
| **AssignedTo** | Person | No | Allow multiple selections: No |
| **CreatedByFlow** | Yes/No | No | Default: Yes |
| **ActualCompletionDate** | Date and time | No | Include time: Yes |
| **Notes** | Multiple lines of text | No | Type: Plain text |
| **AlertTimestamp** | Date and time | No | Include time: Yes |

**Choice Column Details:**

For **Priority** column with color coding:
- Critical → Red background
- High → Orange background
- Medium → Yellow background
- Low → Gray background

To add colors: Edit column → Format this column → Choose format → Use conditional formatting

**1.4. Configure List Settings (Optional but Recommended)**
1. Go to list **Settings** (gear icon → List settings)
2. **Versioning settings**: Enable version history
3. **Advanced settings**: Allow management of content types

---

### **Step 2B: Update Power Automate Flow to Create SharePoint List Items**

Go back to your **Critical Machine Replacement Alert** flow from Part 13 and add SharePoint item creation.

**2.1. Add New Action After Email Alert**

1. In your Power Automate flow, after the **Send an email** action, click **+ New step**
2. Search for **SharePoint**
3. Select **Create item** (SharePoint)

**2.2. Configure Create Item Action**

1. **Site Address**: Select or enter your SharePoint site URL
   - Example: `https://yourtenant.sharepoint.com/sites/maintenance`
2. **List Name**: Select `Work Orders`
3. **Show advanced options**: Click to expand

**Fill in the following fields:**

| Field | Value (Use Dynamic Content) |
|-------|----------------------------|
| **Title** | Enter: `WO-[MachineID]-[formatDateTime(utcNow(), 'yyyyMMdd')]` (e.g., WO-MACHINE-042-20251119) |
| **MachineID** | MachineID (from trigger) |
| **WorkOrderType** | Preventive Maintenance |
| **Priority** | Critical |
| **Status** | New |
| **PredictedReplacementDate** | Pred_ReplacementDate |
| **DaysRemaining** | Pred_DaysLeft |
| **RemainingRotations** | Pred_RemainingRotations |
| **CurrentRotations** | CumulativeBitRotation |
| **CreatedByFlow** | Yes |
| **AlertTimestamp** | PredictionTimestamp |
| **Notes** | Enter text: `Auto-generated work order from ML prediction. Risk Level: [RiskLevel]` |

**Note on Title field:** Use expression builder to create unique title:
```
concat('WO-', triggerOutputs()?['body/MachineID'], '-', formatDateTime(utcNow(), 'yyyyMMdd-HHmm'))
```

3. **Save** the flow

**Flow Architecture Now:**

```
Trigger: When row added to replacement_predictions (IsCritical eq 1)
  └─ Send Email Alert → Maintenance Team
  └─ Create item → Work Orders SharePoint List ✨ NEW
  └─ Post Teams Message (optional)
```

---

### **Step 3B: Create Power Apps Canvas App from SharePoint List**

**3.1. Create App from SharePoint List**

**Method 1: From SharePoint (Easiest)**
1. Go to your SharePoint site
2. Open the **Work Orders** list
3. In the command bar, click **Integrate** → **Power Apps** → **Create an app**
4. Enter app name: `Maintenance Work Orders`
5. Click **Create**

Power Apps automatically generates a three-screen app connected to your SharePoint list.

**Method 2: From Power Apps Portal**
1. Go to https://make.powerapps.com
2. Click **+ Create** → **Blank app** → **Blank canvas app**
3. Name: `Maintenance Work Orders`, Format: Tablet or Phone
4. Click **Create**
5. In the app, click **Data** (left sidebar) → **+ Add data**
6. Search for **SharePoint**
7. Select your SharePoint connection
8. Enter site URL: `https://yourtenant.sharepoint.com/sites/maintenance`
9. Select **Work Orders** list
10. Click **Connect**

**3.2. Build Screens (if using Method 2) or Customize Generated App**

If you used Method 1, Power Apps creates these screens automatically:
- **BrowseScreen1**: List all work orders (BrowseGallery1)
- **DetailScreen1**: View work order details (DetailForm1)
- **EditScreen1**: Edit/create work orders (EditForm1)

Continue to Step 3B.3 for customization.

---

### **Step 3B.3: Customize Browse Screen (Work Order List)**

1. Select **BrowseScreen1** from Tree View
2. Select **BrowseGallery1** (the gallery showing work orders)

**Customize Gallery Layout:**

1. In Properties pane → **Layout**: Select **Title, subtitle, and body**
2. Click **Edit** next to Fields
3. Configure fields:
   - **Title**: `Title` (e.g., WO-MACHINE-042-20251119)
   - **Subtitle**: `MachineID`
   - **Body**: `Status`

**Add Priority Badge:**
- Insert **Label** control inside gallery template (first item)
- **Text**: `ThisItem.Priority.Value`
- **Fill** (background color):
  ```power
  Switch(ThisItem.Priority.Value,
      "Critical", ColorValue("#DC3545"),
      "High", ColorValue("#FD7E14"),
      "Medium", ColorValue("#FFC107"),
      ColorValue("#6C757D")
  )
  ```
- **Color**: White
- Position in upper-right corner of gallery item

**Add Status Badge:**
- Insert **Label** control
- **Text**: `ThisItem.Status.Value`
- **Fill**:
  ```power
  Switch(ThisItem.Status.Value,
      "New", ColorValue("#007BFF"),
      "In Progress", ColorValue("#FFC107"),
      "Completed", ColorValue("#28A745"),
      ColorValue("#6C757D")
  )
  ```
- **Color**: White

**Add Filter Dropdown:**

1. Insert → Input → **Dropdown**
2. **Items**: 
   ```power
   ["All", "New", "In Progress", "Completed"]
   ```
3. **Default**: "All"
4. Position at top of screen

5. Update Gallery **Items** property:
   ```power
   SortByColumns(
       If(Dropdown1.Selected.Value = "All",
          'Work Orders',
          Filter('Work Orders', Status.Value = Dropdown1.Selected.Value)
       ),
       "AlertTimestamp", Descending
   )
   ```

**Add Search Functionality:**

If search box not present, add one:
1. Insert → Input → **Text input**
2. **HintText**: `"Search by Machine ID..."`
3. Update Gallery **Items**:
   ```power
   SortByColumns(
       Filter(
           If(Dropdown1.Selected.Value = "All",
              'Work Orders',
              Filter('Work Orders', Status.Value = Dropdown1.Selected.Value)
           ),
           StartsWith(MachineID, TextInput1.Text) || IsBlank(TextInput1.Text)
       ),
       "AlertTimestamp", Descending
   )
   ```

---

### **Step 3B.4: Customize Detail Screen (View Work Order)**

1. Select **DetailScreen1** from Tree View
2. Select **DetailForm1** (the display form)

**Configure Display Form:**

1. **DataSource**: `'Work Orders'`
2. **Item**: `BrowseGallery1.Selected`

**Add/Reorder Fields:**

1. In Properties pane → **Edit fields**
2. Add all important fields in this order:
   - Title
   - MachineID
   - WorkOrderType
   - Priority
   - Status
   - DaysRemaining
   - PredictedReplacementDate
   - CurrentRotations
   - RemainingRotations
   - AssignedTo
   - AlertTimestamp
   - ActualCompletionDate
   - Notes
3. Click outside to close

**Add Conditional Formatting:**

1. Select **Priority data card** in form
2. Find the label control showing priority value
3. **Color** property:
   ```power
   Switch(Parent.Default.Value,
       "Critical", ColorValue("#DC3545"),
       ColorValue("#000000")
   )
   ```

---

### **Step 3B.5: Customize Edit Screen (Update Work Order)**

1. Select **EditScreen1** from Tree View
2. Select **EditForm1** (the edit form)

**Configure Edit Form:**

1. **DataSource**: `'Work Orders'`
2. **Item**: `BrowseGallery1.Selected`
3. **DefaultMode**: `FormMode.Edit`

**Make Certain Fields Read-Only:**

For auto-generated fields, lock them:
1. Select data card for **Title**
2. **DisplayMode**: `DisplayMode.View`
3. Repeat for:
   - MachineID
   - PredictedReplacementDate
   - DaysRemaining
   - CurrentRotations
   - RemainingRotations
   - CreatedByFlow
   - AlertTimestamp

**Keep These Editable:**
- Status (dropdown)
- AssignedTo (people picker)
- ActualCompletionDate (date picker)
- Notes (text input)

**Configure Save Button:**

Find the **Save icon** (IconAccept) in top-right:
- **OnSelect** property should be:
  ```power
  SubmitForm(EditForm1); Back()
  ```
- **DisplayMode**:
  ```power
  If(EditForm1.Unsaved, DisplayMode.Edit, DisplayMode.Disabled)
  ```

---

### **Step 3B.6: Add Action Buttons**

**A. "Assign to Me" Button (DetailScreen1):**

1. Insert **Button** control
2. **Text**: "Assign to Me"
3. **OnSelect**:
   ```power
   Patch('Work Orders',
       BrowseGallery1.Selected,
       {AssignedTo: {
           '@odata.type': "#Microsoft.Azure.Connectors.SharePoint.SPListExpandedUser",
           Claims: "i:0#.f|membership|" & User().Email,
           DisplayName: User().FullName,
           Email: User().Email,
           Picture: ""
       }}
   );
   Refresh('Work Orders');
   ```
4. **Visible**: 
   ```power
   IsBlank(DetailForm1.LastSubmit.AssignedTo)
   ```

**B. "Start Work" Button (DetailScreen1):**

1. Insert **Button** control
2. **Text**: "Start Work"
3. **OnSelect**:
   ```power
   Patch('Work Orders',
       BrowseGallery1.Selected,
       {
           Status: {Value: "In Progress"},
           AssignedTo: {
               '@odata.type': "#Microsoft.Azure.Connectors.SharePoint.SPListExpandedUser",
               Claims: "i:0#.f|membership|" & User().Email,
               DisplayName: User().FullName,
               Email: User().Email,
               Picture: ""
           }
       }
   );
   Refresh('Work Orders');
   Back();
   ```
4. **Visible**: 
   ```power
   DetailForm1.LastSubmit.Status.Value = "New" ||
   DetailForm1.LastSubmit.Status.Value = "Assigned"
   ```

**C. "Complete Work Order" Button (EditScreen1):**

1. Insert **Button** control
2. **Text**: "Mark as Completed"
3. **Fill**: Green (`ColorValue("#28A745")`)
4. **OnSelect**:
   ```power
   Patch('Work Orders',
       BrowseGallery1.Selected,
       {
           Status: {Value: "Completed"},
           ActualCompletionDate: Now()
       }
   );
   SubmitForm(EditForm1);
   Back();
   ```
5. **Visible**: 
   ```power
   EditForm1.Mode = FormMode.Edit &&
   (EditForm1.LastSubmit.Status.Value = "In Progress" ||
    EditForm1.LastSubmit.Status.Value = "Assigned")
   ```

**Note on SharePoint People Fields:**

When patching Person fields in SharePoint, use the special format shown above with `@odata.type` and Claims. This is different from Dataverse which uses simpler `User()` syntax.

---

### **Step 3B.7: Save and Publish App**

1. **File** → **Save**
2. **App name**: `Maintenance Work Orders`
3. Click **Save**
4. Click **Publish** → **Publish this version**

---

### **Step 4: Test the Complete Workflow**

**End-to-End Test:**

1. **Trigger ML Prediction:**
   - Run notebook Part 11 (daily scoring pipeline)
   - OR manually insert test record with `IsCritical = 1` into Lakehouse table

2. **Verify Power Automate Flow:**
   - Go to Power Automate → My flows
   - Check run history for "Critical Machine Replacement Alert"
   - Verify:
     ✅ Email sent to maintenance team
     ✅ Work order created in **Dataverse** (Option A) OR **SharePoint List** (Option B)

3. **Open Power Apps:**
   - Go to https://make.powerapps.com → Apps
   - Click **Maintenance Work Orders** app
   - Verify new work order appears in list
   - Status should be "New"
   - Priority should be "Critical" (red badge)

4. **Assign Work Order:**
   - Click on work order to open details (DetailScreen)
   - Review all fields (Machine ID, Days Remaining, Predicted Date)
   - Click **Assign to Me** button
   - Verify "Assigned To" shows your name

5. **Start Work:**
   - Click **Start Work** button
   - Status changes to "In Progress" (yellow badge)
   - Return to browse screen and verify status updated

6. **Update Work Order:**
   - Select the work order again
   - Click **Edit** icon (pencil in top-right)
   - Add notes in Notes field: "Replaced drill bit, tested machine operation"
   - Click **Complete Work Order** button
   - Status changes to "Completed" (green badge)
   - Actual Completion Date is automatically set to current date/time

7. **Verify in Backend:**

   **For Dataverse (Option A):**
   - Go to Power Apps → Tables → Work Orders
   - Click **Data** tab
   - Verify all fields are populated correctly
   - Check Modified By and Modified On columns

   **For SharePoint (Option B):**
   - Go to your SharePoint site
   - Open Work Orders list
   - Verify new item appears with all fields populated
   - Check Created By and Modified By columns
   - View version history (if enabled)

---

### **Step 5: Share App with Maintenance Team**

1. In Power Apps, open your app
2. Click **Share** (top-right)
3. Enter email addresses or groups:
   - `maintenance-team@yourcompany.com`
   - Individual engineers
4. **Permission**: Select **Can use** (not "Can edit")
5. **Send email invitation**: Check box
6. Click **Share**

**Users can access app via:**
- Email invitation link
- Power Apps mobile app (iOS/Android)
- https://make.powerapps.com → Apps → Maintenance Work Orders

---

### **📊 Complete Architecture:**

**Option A: Dataverse Backend**
```
ML Model (Fabric Notebook)
  │
  ├─> Prediction Table (Lakehouse)
  │     │
  │     └─> Power Automate (Dataverse Trigger)
  │           │
  │           ├─> Send Email Alert
  │           └─> Add a new row → Work Orders (Dataverse Table)
  │                 │
  │                 └─> Power Apps (Canvas App)
  │                       │
  │                       ├─> Browse Work Orders
  │                       ├─> View Details
  │                       ├─> Assign to Engineer
  │                       ├─> Update Status
  │                       └─> Complete Work Order
  │
  └─> Power BI Dashboard (Real-time monitoring)
```

**Option B: SharePoint List Backend**
```
ML Model (Fabric Notebook)
  │
  ├─> Prediction Table (Lakehouse)
  │     │
  │     └─> Power Automate (Dataverse Trigger)
  │           │
  │           ├─> Send Email Alert
  │           └─> Create item → Work Orders (SharePoint List)
  │                 │
  │                 └─> Power Apps (Canvas App)
  │                       │
  │                       ├─> Browse Work Orders
  │                       ├─> View Details
  │                       ├─> Assign to Engineer
  │                       ├─> Update Status
  │                       └─> Complete Work Order
  │
  └─> Power BI Dashboard (Real-time monitoring)
```

---

### **🎯 Key Features:**

✅ **Automated Work Order Creation** - No manual entry required  
✅ **Real-Time Alerts** - Engineers notified immediately via email  
✅ **Mobile Access** - Update status from factory floor using Power Apps mobile  
✅ **Status Tracking** - New → Assigned → In Progress → Completed  
✅ **Assignment Management** - Self-assign or assign to specific engineers  
✅ **Completion Timestamp** - Automatically track actual maintenance times  
✅ **Notes Field** - Document work performed and findings  
✅ **Priority Color-Coding** - Visual indicators (Red=Critical, Orange=High, Yellow=Medium)  
✅ **Filter & Search** - Quickly find relevant work orders by status or machine ID  
✅ **Flexible Backend** - Choose between Dataverse or SharePoint List

---

### **Backend Comparison Summary:**

| Aspect | Dataverse | SharePoint List |
|--------|-----------|----------------|
| **Setup Time** | 15-20 minutes | 10-15 minutes |
| **License Cost** | Premium required | Included with M365 |
| **Data Capacity** | Unlimited (GB-TB scale) | 25 million items per tenant |
| **Complex Relationships** | Yes, full relational | Limited via lookups |
| **Business Rules** | Advanced validation rules | Basic column validation |
| **Power Apps Performance** | Faster with delegation | Good for <2K items displayed |
| **Offline Sync** | Native support | Via Power Apps offline |
| **Audit Trail** | Built-in with tracking | Version history |
| **Best Use Case** | Enterprise-scale deployments | Departmental quick wins |

**When to switch:** If you start with SharePoint and later need advanced features (complex relationships, business process flows, advanced security), migrate to Dataverse using Power Automate or Power Query.

---

### **📱 Power Apps Mobile App Setup (Optional):**

1. Install **Power Apps** from App Store (iOS) or Play Store (Android)
2. Sign in with Microsoft 365 account
3. App appears in app list automatically (no additional setup needed)
4. Engineers can update work orders on the factory floor
5. Works offline with automatic sync when connection restored
6. Push notifications available (configure in app settings)

**Mobile Best Practices:**
- Use **Phone** layout when creating app for better mobile experience
- Test touch targets (buttons at least 48x48 pixels)
- Optimize gallery scrolling performance
- Enable offline mode for areas with poor connectivity

---

### **🔄 Optional Enhancement: Notify Engineer When Assigned**

Add automatic email notification when work order is assigned to an engineer.

**For Dataverse Backend:**

1. Create new Power Automate flow:
   - **Trigger**: When a row is added, modified or deleted (Microsoft Dataverse)
   - **Table name**: Work Orders
   - **Change type**: Modified
   - **Filter rows**: `_assignedto_value ne null`
2. **Condition**: Check if AssignedTo changed (compare trigger old vs new values)
3. **Action**: Send an email (V2)
   - **To**: Dynamic content → Assigned To → Email
   - **Subject**: `New Work Order Assigned: [Work Order ID]`
   - **Body**:
     ```
     Hi [Assigned To DisplayName],
     
     A new work order has been assigned to you:
     
     Work Order ID: [Work Order ID]
     Machine ID: [MachineID]
     Priority: [Priority]
     Days Remaining: [DaysRemaining]
     Predicted Date: [PredictedReplacementDate]
     
     Please review and update the status in the Work Orders app.
     
     Link to app: [Your Power Apps URL]
     ```

**For SharePoint List Backend:**

1. Create new Power Automate flow:
   - **Trigger**: When an item is created or modified (SharePoint)
   - **Site Address**: Your SharePoint site
   - **List Name**: Work Orders
2. **Condition**: `AssignedTo Email is not empty`
3. **Action**: Send an email (V2)
   - **To**: Dynamic content → AssignedTo → Email
   - **Subject**: Same as above
   - **Body**: Same format, use SharePoint dynamic content

---

### **🔍 Troubleshooting Common Issues:**

**Issue: Power Apps doesn't show SharePoint list columns**
- Solution: Refresh data source (Data → ... next to SharePoint → Refresh)

**Issue: "Assign to Me" button doesn't work with SharePoint**
- Solution: Use the specific format for Person fields shown in Step 3B.6

**Issue: Flow runs but no work order created**
- Solution: Check flow run history for errors, verify table/list name spelling

**Issue: Power Apps shows delegation warning**
- Solution: Keep filtered data under 2,000 items or use collection for offline data

**Issue: Cannot edit Choice fields in Power Apps (SharePoint)**
- Solution: Use `.Value` property (e.g., `ThisItem.Status.Value` instead of `ThisItem.Status`)

**Issue: Dataverse table not showing in Power Automate**
- Solution: Verify you're in correct environment, wait 5 minutes for sync, refresh page


## Part 15: Model Monitoring and Retraining Strategy

**Retraining Triggers:**

1. **Scheduled retrain:** Weekly (first month) → Monthly (after stable)
2. **Data drift detection:** When feature distributions shift significantly
3. **Performance degradation:** When MAE increases beyond threshold (e.g., +20%)
4. **New replacement events:** After every 10-20 new completed cycles

**Monitoring Metrics:**

- Prediction MAE (rotations and days)
- Feature drift scores
- Prediction error over time
- Actual vs predicted replacement dates

In [ ]:
# Model monitoring: track prediction errors over time
def check_model_performance():
    """
    Check model performance on recent data
    Compare predicted vs actual for completed cycles
    """
    
    # Load recent predictions
    recent_predictions = spark.read.table("MaintenanceML.replacement_predictions").toPandas()
    recent_predictions['PredictionTimestamp'] = pd.to_datetime(recent_predictions['PredictionTimestamp'])
    
    # Get predictions from last 7 days
    last_week = pd.Timestamp.now() - pd.Timedelta(days=7)
    recent = recent_predictions[recent_predictions['PredictionTimestamp'] >= last_week]
    
    print(f"📊 Model Monitoring Report")
    print(f"Period: Last 7 days")
    print(f"Total predictions: {len(recent)}")
    print(f"\nRisk Distribution:")
    print(recent['RiskLevel'].value_counts())
    print(f"\nAverage predicted days left: {recent['Pred_DaysLeft'].mean():.1f}")
    print(f"Critical alerts: {recent['IsCritical'].sum()}")
    
    # TODO: Compare with actual replacement events
    # Calculate actual MAE for machines that were replaced
    
    return recent

# Run monitoring check
monitoring_report = check_model_performance()

## Part 15: Summary and Next Steps

### ✅ What We Built:

1. **Replacement Event Detection:** Automatic detection of bit changes via counter resets
2. **Cycle Segmentation:** Split data into per-replacement cycles
3. **Target Labeling:** Calculate remaining rotations until next replacement
4. **Feature Engineering:** Rolling windows, time features, rotation rates
5. **XGBoost Model:** Predict remaining rotations with MAE tracking
6. **Date Conversion:** Convert rotation predictions to replacement dates
7. **Risk Levels:** 4-tier classification (CRITICAL/HIGH/MEDIUM/LOW)
8. **MLflow Tracking:** Full experiment tracking and model versioning
9. **Production Pipeline:** Daily scoring workflow
10. **Power BI Integration:** Real-time dashboard with DAX measures
11. **Alerting:** Teams/email notifications for critical machines

### 📈 Model Performance:

- **Validation MAE (rotations):** Check Part 6 output
- **Validation MAE (days):** Check Part 10 output
- **R² Score:** Check Part 6 output

### 🚀 Next Steps:

1. **Deploy to Production:**
   - Schedule daily notebook run in Fabric Data Pipeline
   - Set up Power Automate alerts
   - Create Power BI dashboard

2. **Model Improvements:**
   - Add SHAP explanations for predictions
   - Experiment with survival models (Cox regression)
   - Add more sensor features (temperature, vibration)
   - Tune XGBoost hyperparameters

3. **Operational Integration:**
   - Link to work order system (Dynamics 365)
   - Track actual replacement dates
   - Calculate cost savings from predictive maintenance
   - Implement feedback loop for model improvement

4. **Monitoring:**
   - Set up data drift detection
   - Track prediction accuracy over time
   - Establish retrain cadence
   - Monitor alert effectiveness

### 📚 Key Learnings:

- **Counter resets are your friend:** They define natural cycle boundaries
- **Time-based splits matter:** Avoid temporal leakage in validation
- **Rotation rate is crucial:** Converting rotations to days requires accurate rate estimates
- **Risk levels drive action:** Classification helps prioritize maintenance
- **Continuous learning:** Model improves as more replacement cycles complete

---

**Created with Microsoft Fabric ML**  
**Model:** XGBoost Regression  
**Target:** Predict next bit replacement date  
**Deployment:** Daily batch scoring with adaptive retraining